In [14]:
"""
This version of code is modified to incorporate more batch operations by Xiaofeng. 
The following parts are covered: Error computations, argmax subproblems, nonlinear Newton solver
Oct 13rd 2024.
"""

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import time
import sys
import os 
from scipy.sparse import linalg
from pathlib import Path
import itertools
if torch.cuda.is_available():  
    device = "cuda" 
else:  
    device = "cpu" 

torch.set_default_dtype(torch.float64)
pi = torch.tensor(np.pi,dtype=torch.float64)
ZERO = torch.tensor([0.]).to(device)


class model(nn.Module):
    """ ReLU k shallow neural network
    Parameters: 
    input size: input dimension
    hidden_size1 : number of hidden layers 
    num_classes: output classes 
    k: degree of relu functions
    """
    def __init__(self, input_size, hidden_size1, num_classes,k = 1):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, num_classes,bias = False)
        self.k = k 
    def forward(self, x):
        u1 = self.fc2(F.relu(self.fc1(x))**self.k)
        return u1
    def evaluate_derivative(self, x, i):
        if self.k == 1:
            u1 = self.fc2(torch.heaviside(self.fc1(x),ZERO) * self.fc1.weight.t()[i-1:i,:] )
        else:
            u1 = self.fc2(self.k*F.relu(self.fc1(x))**(self.k-1) *self.fc1.weight.t()[i-1:i,:] )  
        return u1

def plot_2D(f): 
    
    Nx = 400
    Ny = 400 
    xs = np.linspace(0, 1, Nx)
    ys = np.linspace(0, 1, Ny)
    x, y = np.meshgrid(xs, ys, indexing='xy')
    xy_comb = np.stack((x.flatten(),y.flatten())).T
    xy_comb = torch.tensor(xy_comb)
    z = f(xy_comb).reshape(Nx,Ny)
    z = z.detach().numpy()
    plt.figure(dpi=200)
    ax = plt.axes(projection='3d')
    ax.plot_surface(x , y , z )

    plt.show()

def plot_subdomains(my_model):
    x_coord =torch.linspace(0,1,200)
    wi = my_model.fc1.weight.data
    bi = my_model.fc1.bias.data 
    for i, bias in enumerate(bi):  
        if wi[i,1] !=0: 
            plt.plot(x_coord, - wi[i,0]/wi[i,1]*x_coord - bias/wi[i,1])
        else: 
            plt.plot(x_coord,  - bias/wi[i,0]*torch.ones(x_coord.size()))

    plt.xlim([0,1])
    plt.ylim([0,1])
    plt.legend()
    plt.show()
    return 0   

def adjust_neuron_position(my_model, dims = 3):

    def create_mesh_grid(dims, pts):
        mesh = torch.tensor(list(itertools.product(pts,repeat=dims)))
        vertices = mesh.reshape(len(pts) ** dims, -1) 
        return vertices
    counter = 0 
    # positions = torch.tensor([[0.,0.],[0.,1.],[1.,1.],[1.,0.]])
    pts = torch.tensor([0.,1.])
    positions = create_mesh_grid(dims,pts) 
    neuron_num = my_model.fc1.bias.size(0)
    for i in range(neuron_num): 
        w = my_model.fc1.weight.data[i:i+1,:]
        b = my_model.fc1.bias.data[i]
    #     print(w,b)
        values = torch.matmul(positions,w.T) # + b
        left_end = - torch.max(values)
        right_end = - torch.min(values)
        offset = (right_end - left_end)/50
        if b <= left_end + offset/2 : 
            b = torch.rand(1)*(right_end - left_end - offset) + left_end + offset/2 
            my_model.fc1.bias.data[i] = b 
        if b >= right_end - offset/2 :
            if counter < (dims+1):
#                 print("here")
                counter += 1
            else: # (d + 1) or more 
                b = torch.rand(1)*(right_end - left_end - offset) + left_end + offset/2 
                my_model.fc1.bias.data[i] = b 
    return my_model



In [2]:

def show_convergence_order(err_l2,err_h10,exponent,dict_size, filename,write2file = False):
    
    if write2file:
        file_mode = "a" if os.path.exists(filename) else "w"
        f_write = open(filename, file_mode)
    
    neuron_nums = [2**j for j in range(2,exponent+1)]
    err_list = [err_l2[i] for i in neuron_nums ]
    err_list2 = [err_h10[i] for i in neuron_nums ] 
    # f_write.write('M:{}, relu {} \n'.format(M,k))
    if write2file:
        f_write.write('dictionary size: {}\n'.format(dict_size))
        f_write.write("neuron num \t\t error \t\t order \t\t h10 error \\ order \n")
    print("neuron num \t\t error \t\t order")
    for i, item in enumerate(err_list):
        if i == 0: 
            # print(neuron_nums[i], end = "\t\t")
            # print(item, end = "\t\t")
            
            # print("*")
            print("{} \t\t {:.6f} \t\t * \t\t {:.6f} \t\t * \n".format(neuron_nums[i],item, err_list2[i] ) )
            if write2file: 
                f_write.write("{} \t\t {} \t\t * \t\t {} \t\t * \n".format(neuron_nums[i],item, err_list2[i] ))
        else: 
            # print(neuron_nums[i], end = "\t\t")
            # print(item, end = "\t\t") 
            # print(np.log(err_list[i-1]/err_list[i])/np.log(2))
            print("{} \t\t {:.6f} \t\t {:.6f} \t\t {:.6f} \t\t {:.6f} \n".format(neuron_nums[i],item,np.log(err_list[i-1]/err_list[i])/np.log(2),err_list2[i] , np.log(err_list2[i-1]/err_list2[i])/np.log(2) ) )
            if write2file: 
                f_write.write("{} \t\t {} \t\t {} \t\t {} \t\t {} \n".format(neuron_nums[i],item,np.log(err_list[i-1]/err_list[i])/np.log(2),err_list2[i] , np.log(err_list2[i-1]/err_list2[i])/np.log(2) ))
    if write2file:     
        f_write.write("\n")
        f_write.close()

def show_convergence_order_latex(err_l2,err_h10,exponent,k=1,d=1): 
    neuron_nums = [2**j for j in range(2,exponent+1)]
    err_list = [err_l2[i] for i in neuron_nums ]
    err_list2 = [err_h10[i] for i in neuron_nums ] 
    # f_write.write('M:{}, relu {} \n'.format(M,k))
    # f_write.write('randomized dictionary size: {}\n'.format(N))
    # f_write.write("neuron num \t\t error \t\t order \t\t h10 error \\ order \n")
    l2_order = -1/2-(2*k + 1)/(2*d)
    h10_order = -1/2-(2*(k-1) + 1)/(2*d)
#     print("neuron num  & \t $\|u-u_n \|_{L^2}$ & \t order $O(n^{{{}})$ & \t $ | u -u_n |_{H^1}$ & \t order $O(n^{{{}})$ \\\ \hline \hline ".format(l2_order,h10_order))
    print("neuron num  & \t $\\|u-u_n \\|_{{L^2}}$ & \t order $O(n^{{{:.2f}}})$ & \t $ | u -u_n |_{{H^1}}$ & \t order $O(n^{{{:.2f}}})$ \\\\ \\hline \\hline ".format(l2_order, h10_order))
    for i, item in enumerate(err_list):
        if i == 0: 
            # print(neuron_nums[i], end = "\t\t")
            # print(item, end = "\t\t")

            # print("*")
            print("{} \t\t & {:.6f} &\t\t * & \t\t {:.6f} & \t\t *  \\\ \hline  \n".format(neuron_nums[i],item, err_list2[i] ) )   
            # f_write.write("{} \t\t {} \t\t * \t\t {} \t\t * \n".format(neuron_nums[i],item, err_list2[i] ))
        else: 
            # print(neuron_nums[i], end = "\t\t")
            # print(item, end = "\t\t") 
            # print(np.log(err_list[i-1]/err_list[i])/np.log(2))
            print("{} \t\t &  {:.3e} &  \t\t {:.2f} &  \t\t {:.3e} &  \t\t {:.2f} \\\ \hline  \n".format(neuron_nums[i],item,np.log(err_list[i-1]/err_list[i])/np.log(2),err_list2[i] , np.log(err_list2[i-1]/err_list2[i])/np.log(2) ) )
            # f_write.write("{} \t\t {} \t\t {} \t\t {} \t\t {} \n".format(neuron_nums[i],item,np.log(err_list[i-1]/err_list[i])/np.log(2),err_list2[i] , np.log(err_list2[i-1]/err_list2[i])/np.log(2) ))
    # f_write.write("\n")
    # f_write.close()

In [3]:
def PiecewiseGQ1D_weights_points(x_l,x_r,Nx, order):
    """ Output the coeffients and weights for piecewise Gauss Quadrature 
    Parameters
    ----------
    x_l : float 
    left endpoint of an interval 
    x_r: float
    right endpoint of an interval 
    Nx: int 
    number of subintervals for integration
    order: int
    order of Gauss Quadrature 
    Returns
    -------
    vectorized quadrature weights and integration points
    """
    x,w = np.polynomial.legendre.leggauss(order)
    gx = torch.tensor(x).to(device)
    gx = gx.view(1,-1) # row vector 
    gw = torch.tensor(w).to(device)    
    gw = gw.view(-1,1) # Column vector 
    nodes = torch.linspace(x_l,x_r,Nx+1).view(-1,1).to(device) 
    coef1 = ((nodes[1:,:] - nodes[:-1,:])/2) # n by 1  
    coef2 = ((nodes[1:,:] + nodes[:-1,:])/2) # n by 1  
    coef2_expand = coef2.expand(-1,gx.size(1)) # Expand to n by p shape, -1: keep the first dimension n , expand the 2nd dim (columns)
    integration_points = coef1@gx + coef2_expand
    integration_points = integration_points.flatten().view(-1,1) # Make it a column vector
    gw_expand = torch.tile(gw,(Nx,1)) # rows: n copies of current tensor, columns: 1 copy, no change
    # Modify coef1 to be compatible with func_values
    coef1_expand = coef1.expand(coef1.size(0),gx.size(1))    
    coef1_expand = coef1_expand.flatten().view(-1,1)
    return coef1_expand.to(device) * gw_expand.to(device), integration_points.to(device)

def PiecewiseGQ2D_weights_points(Nx, order): 
    """ A slight modification of PiecewiseGQ2D function that only needs the weights and integration points.
    Parameters
    ----------

    Nx: int 
        number of intervals along the dimension. No Ny, assume Nx = Ny
    order: int 
        order of the Gauss Quadrature

    Returns
    -------
    long_weights: torch.tensor
    integration_points: torch.tensor
    """

#     print("order: ",order )
    x, w = np.polynomial.legendre.leggauss(order)
    gauss_pts = np.array(np.meshgrid(x,x,indexing='ij')).reshape(2,-1).T
    weights =  (w*w[:,None]).ravel()

    gauss_pts =torch.tensor(gauss_pts)
    weights = torch.tensor(weights)

    h = 1/Nx # 100 intervals 
    long_weights =  torch.tile(weights,(Nx**2,1))
    long_weights = long_weights.reshape(-1,1)
    long_weights = long_weights * h**2 /4 

    integration_points = torch.tile(gauss_pts,(Nx**2,1))
    scale_factor = h/2 
    integration_points = scale_factor * integration_points

    index = np.arange(1,Nx+1)-0.5
    ordered_pairs = np.array(np.meshgrid(index,index,indexing='ij'))
    ordered_pairs = ordered_pairs.reshape(2,-1).T

    # print(ordered_pairs)
    # print()
    ordered_pairs = torch.tensor(ordered_pairs)
    # print(ordered_pairs.size())
    ordered_pairs = torch.tile(ordered_pairs, (1,order**2)) # number of GQ points
    # print(ordered_pairs)

    ordered_pairs =  ordered_pairs.reshape(-1,2)
    # print(ordered_pairs)
    translation = ordered_pairs*h 
    # print(translation)

    integration_points = integration_points + translation 
#     print(integration_points.size())
    # func_values = integrand2_torch(integration_points)
    return long_weights.to(device), integration_points.to(device)

def PiecewiseGQ3D_weights_points(Nx, order): 
    """ A slight modification of PiecewiseGQ2D function that only needs the weights and integration points.
    Parameters
    ----------

    Nx: int 
        number of intervals along the dimension. No Ny, assume Nx = Ny
    order: int 
        order of the Gauss Quadrature

    Returns
    -------
    long_weights: torch.tensor
    integration_points: torch.tensor
    """

    """
    Parameters
    ----------
    target : 
        Target function 
    Nx: int 
        number of intervals along the dimension. No Ny, assume Nx = Ny
    order: int 
        order of the Gauss Quadrature
    """

    # print("order: ",order )
    x, w = np.polynomial.legendre.leggauss(order)
    gauss_pts = np.array(np.meshgrid(x,x,x,indexing='ij')).reshape(3,-1).T
    weight_list = np.array(np.meshgrid(w,w,w,indexing='ij'))
    weights =   (weight_list[0]*weight_list[1]*weight_list[2]).ravel() 

    gauss_pts =torch.tensor(gauss_pts)
    weights = torch.tensor(weights)

    h = 1/Nx # 100 intervals 
    long_weights =  torch.tile(weights,(Nx**3,1))
    long_weights = long_weights.reshape(-1,1)
    long_weights = long_weights * h**3 /8 

    integration_points = torch.tile(gauss_pts,(Nx**3,1))
    # print("shape of integration_points", integration_points.size())
    scale_factor = h/2 
    integration_points = scale_factor * integration_points

    index = np.arange(1,Nx+1)-0.5
    ordered_pairs = np.array(np.meshgrid(index,index,index,indexing='ij'))
    ordered_pairs = ordered_pairs.reshape(3,-1).T

    # print(ordered_pairs)
    # print()
    ordered_pairs = torch.tensor(ordered_pairs)
    # print(ordered_pairs.size())
    ordered_pairs = torch.tile(ordered_pairs, (1,order**3)) # number of GQ points
    # print(ordered_pairs)

    ordered_pairs =  ordered_pairs.reshape(-1,3)
    # print(ordered_pairs)
    translation = ordered_pairs*h 
    # print(translation)

    integration_points = integration_points + translation 

    return long_weights.to(device), integration_points.to(device)

def MonteCarlo_Sobol_dDim_weights_points(M ,d = 4):
    Sob_integral = torch.quasirandom.SobolEngine(dimension =d, scramble= False, seed=None) 
    integration_points = Sob_integral.draw(M).double() 
    integration_points = integration_points.to(device)
    weights = torch.ones(M,1).to(device)/M 
    return weights, integration_points 

def Neumann_boundary_quadrature_points_weights(M,d):
    def generate_quadpts_on_boundary(gw_expand_bd, integration_points_bd,d):
        size_pts_bd = integration_points_bd.size(0) 
        gw_expand_bd_faces = torch.tile(gw_expand_bd,(2*d,1)) # 2d boundaries, 拉成长条

        integration_points_bd_faces = torch.zeros(2*d*integration_points_bd.size(0),d).to(device)
        for ind in range(d): 
            integration_points_bd_faces[2 *ind * size_pts_bd :(2 *ind +1) * size_pts_bd,ind:ind+1] = 0 
            integration_points_bd_faces[(2 *ind)*size_pts_bd :(2 * ind +1) * size_pts_bd,:ind] = integration_points_bd[:,:ind]
            integration_points_bd_faces[(2 *ind)*size_pts_bd :(2 * ind +1) * size_pts_bd,ind+1:] = integration_points_bd[:,ind:]

            integration_points_bd_faces[(2 *ind +1) * size_pts_bd:(2 *ind +2)*size_pts_bd,ind:ind+1] = 1
            integration_points_bd_faces[(2 *ind +1) * size_pts_bd:(2 *ind +2)*size_pts_bd,:ind] = integration_points_bd[:,:ind]        
            integration_points_bd_faces[(2 *ind +1) * size_pts_bd:(2 *ind +2)*size_pts_bd,ind+1:] = integration_points_bd[:,ind:]
        return gw_expand_bd_faces, integration_points_bd_faces
    
    if d == 1: 
        print('dim',d)
        gw_expand_bd_faces = torch.tensor([1.,1.]).view(-1,1).to(device)
        integration_points_bd_faces = torch.tensor([0.,1.]).view(-1,1).to(device) 
    elif d == 2: 
        print('dim',d)
        gw_expand_bd, integration_points_bd = PiecewiseGQ1D_weights_points(0,1,8192, order = 3) 
    elif d == 3: 
        gw_expand_bd, integration_points_bd = PiecewiseGQ2D_weights_points(200, order = 3) 
    elif d == 4: 
        gw_expand_bd, integration_points_bd = PiecewiseGQ3D_weights_points(25, order = 3) 
        print('dim',d)
    else: 
        gw_expand_bd, integration_points_bd = MonteCarlo_Sobol_dDim_weights_points(M ,d = d)
        print('dim >=5 ')
    gw_expand_bd_faces, integration_points_bd_faces = generate_quadpts_on_boundary(gw_expand_bd, integration_points_bd,d)
    return gw_expand_bd_faces.to(device), integration_points_bd_faces.to(device) 

def generate_relu_dict3D(N_list):
    N1 = N_list[0]
    N2 = N_list[1]
    N3 = N_list[2]
    
    N = N1*N2*N3 
    theta1 = np.linspace(0, pi, N1, endpoint= True).reshape(N1,1)
    theta2 = np.linspace(0, 2*pi, N2, endpoint= False).reshape(N2,1)
    b = np.linspace(-1.732, 1.732, N3,endpoint=False).reshape(N3,1) # threshold: 3**0.5  
    coord3 = np.array(np.meshgrid(theta1,theta2,b,indexing='ij'))
    coord3 = coord3.reshape(3,-1).T # N1*N2*N3 x 3. coordinates for the grid points 
    coord3 = torch.tensor(coord3) 

    f1 = torch.zeros(N,1) 
    f2 = torch.zeros(N,1)
    f3 = torch.zeros(N,1)
    f4 = torch.zeros(N,1)

    f1[:,0] = torch.cos(coord3[:,0]) 
    f2[:,0] = torch.sin(coord3[:,0]) * torch.cos(coord3[:,1])
    f3[:,0] = torch.sin(coord3[:,0]) * torch.sin(coord3[:,1])
    f4[:,0] = coord3[:,2] 

    Wb_tensor = torch.cat([f1,f2,f3,f4],1) # N x 4 
    return Wb_tensor


def generate_relu_dict3D_QMC(s,N0):
#     Sob = torch.quasirandom.SobolEngine(dimension =3, scramble= True, seed=None) 
#     samples = Sob.draw(N0).double() 

#     for i in range(s-1):
#         samples = torch.cat([samples,Sob.draw(N0).double()],0)

    # Monte Carlo 
    samples = torch.rand(s*N0,3) 
    T =torch.tensor([[pi,0,0],[0,2*pi,0],[0,0,1.732*2]])
    shift = torch.tensor([0,0,-1.732])
    samples = samples@T + shift 

    f1 = torch.zeros(s*N0,1) 
    f2 = torch.zeros(s*N0,1)
    f3 = torch.zeros(s*N0,1)
    f4 = torch.zeros(s*N0,1)

    f1[:,0] = torch.cos(samples[:,0]) 
    f2[:,0] = torch.sin(samples[:,0]) * torch.cos(samples[:,1])
    f3[:,0] = torch.sin(samples[:,0]) * torch.sin(samples[:,1])
    f4[:,0] = samples[:,2] 

    Wb_tensor = torch.cat([f1,f2,f3,f4],1) # N x 4 
    return Wb_tensor


def minimize_linear_layer_H1_explicit_assemble_efficient(model,alpha, target, g_N, weights, integration_points, w_bd, pts_bd, activation = 'relu',solver="direct" ,memory=2**29):
    """ -div alpha grad u(x) + u = f 
    Parameters
    ----------
    model: 
        nn model
    alpha:
        alpha function
    target:
        rhs function f 
    pts_bd:
        integration points on the boundary, embdedded in the domain 
    """ 
    zero = torch.tensor([0.]).to(device)
    start_time = time.time() 
    w = model.fc1.weight.data 
    b = model.fc1.bias.data 
    neuron_num = b.size(0) 
    dim = integration_points.size(1) 
    M = integration_points.size(0)
    coef_alpha = alpha(integration_points) # alpha  

    total_size = neuron_num * M # memory, number of floating numbers 
    print('total size: {} {} = {}'.format(neuron_num,M,total_size))
    num_batch = total_size//memory + 1 # divide according to memory
    print("num batches: ",num_batch)
    batch_size = M//num_batch
    jac = torch.zeros(b.size(0),b.size(0)).to(device)
    rhs = torch.zeros(b.size(0),1).to(device)
    
    # Assemble the mass matrix <g_j,g_i>_{\Omega} and the rhs <f,g_i>_{\Omega} 
    for j in range(0,M,batch_size): 
        end_index = j + batch_size
        basis_value_col = F.relu(integration_points[j:end_index] @ w.t()+ b)**(model.k) 
        weighted_basis_value_col = basis_value_col * weights[j:end_index] 
        jac += weighted_basis_value_col.t() @ basis_value_col 
        rhs += weighted_basis_value_col.t() @ (target(integration_points[j:end_index,:])) 
        
    # Assemble the boundary condition term <g,v>_{\Gamma_N} 
    if g_N != None: # no batch operations for the boundary part, since it is only rhs on the boundary 
        size_pts_bd = int(pts_bd.size(0)/(2*dim))
        bcs_N = g_N(dim)
        for ii, g_ii in bcs_N:
            # pts_bd_ii = pts_bd[2*ii*size_pts_bd:(2*ii+1)*size_pts_bd,:]
            weighted_g_N = -g_ii(pts_bd[2*ii*size_pts_bd:(2*ii+1)*size_pts_bd,:])* w_bd[2*ii*size_pts_bd:(2*ii+1)*size_pts_bd,:]
            basis_value_bd_col = F.relu(pts_bd[2*ii*size_pts_bd:(2*ii+1)*size_pts_bd,:] @ w.t()+ b)**(model.k)
            rhs += basis_value_bd_col.t() @ weighted_g_N

            weighted_g_N = g_ii(pts_bd[(2*ii+1)*size_pts_bd:(2*ii+2)*size_pts_bd,:])* w_bd[(2*ii+1)*size_pts_bd:(2*ii+2)*size_pts_bd,:]
            basis_value_bd_col = F.relu(pts_bd[(2*ii+1)*size_pts_bd:(2*ii+2)*size_pts_bd,:] @ w.t()+ b)**(model.k)
            rhs += basis_value_bd_col.t() @ weighted_g_N

    # Stiffness matrix term in the jacobian 
    for d in range(dim):
        if model.k == 1:  
            for j in range(0,M,batch_size):  
                end_index = j + batch_size 
                basis_value_dxi_col = torch.heaviside(integration_points[j:end_index] @ w.t()+ b, zero) * w.t()[d:d+1,:]
                weighted_basis_value_dx_col = basis_value_dxi_col * weights[j:end_index] * coef_alpha[j:end_index] 
                jac += weighted_basis_value_dx_col.t() @ basis_value_dxi_col 
#             basis_value_dxi_col = torch.heaviside(integration_points @ w.t()+ b, zero) * w.t()[d:d+1,:]
#             weighted_basis_value_dx_col = basis_value_dxi_col * weights * coef_alpha 
#             jac += weighted_basis_value_dx_col.t() @ basis_value_dxi_col 

        else:
            for j in range(0,M,batch_size):  
                end_index = j + batch_size 
                basis_value_dxi_col = model.k * F.relu(integration_points[j:end_index] @ w.t()+ b)**(model.k-1) * w.t()[d:d+1,:]
                weighted_basis_value_dx_col = basis_value_dxi_col * weights[j:end_index] * coef_alpha[j:end_index] 
                jac += weighted_basis_value_dx_col.t() @ basis_value_dxi_col 
#             basis_value_dxi_col = model.k * F.relu(integration_points @ w.t()+ b)**(model.k-1) * w.t()[d:d+1,:]
#             weighted_basis_value_dx_col = basis_value_dxi_col * weights * coef_alpha  
#             jac += weighted_basis_value_dx_col.t() @ basis_value_dxi_col 

    print("assembling the mass matrix time taken: ", time.time()-start_time) 

    start_time = time.time()    
    if solver == "cg": 
        sol, exit_code = linalg.cg(np.array(jac.detach().cpu()),np.array(rhs.detach().cpu()),tol=1e-12)
        sol = torch.tensor(sol).view(1,-1)
    elif solver == "direct": 
#         sol = np.linalg.inv( np.array(jac.detach().cpu()) )@np.array(rhs.detach().cpu())
        sol = (torch.linalg.solve( jac.detach(), rhs.detach())).view(1,-1)
    elif solver == "ls":
        sol = (torch.linalg.lstsq(jac.detach().cpu(),rhs.detach().cpu(),driver='gelsd').solution).view(1,-1)
        # sol = (torch.linalg.lstsq(jac.detach(),rhs.detach()).solution).view(1,-1) # gpu/cpu, driver = 'gels', cannot solve singular
    print("solving Ax = b time taken: ", time.time()-start_time)
    return sol 


def minimize_linear_layer_explicit_assemble(model,target,weights, integration_points,solver="direct"):
    """
    calls the following functions (dependency): 
    1. GQ_piecewise_2D
    input: the nn model containing parameter 
    1. define the loss function  
    2. take derivative to extract the linear system A
    3. call the cg solver in scipy to solve the linear system 
    output: sol. solution of Ax = b
    """
    start_time = time.time() 
    w = model.fc1.weight.data 
    b = model.fc1.bias.data 
    basis_value_col = F.relu(integration_points @ w.t()+ b)**(model.k) 
    weighted_basis_value_col = basis_value_col * weights 
    jac = weighted_basis_value_col.t() @ basis_value_col 
     
    rhs = weighted_basis_value_col.t() @ (target(integration_points)) 
    print("assembling the matrix time taken: ", time.time()-start_time) 
    start_time = time.time()    
    if solver == "cg": 
        sol, exit_code = linalg.cg(np.array(jac.detach().cpu()),np.array(rhs.detach().cpu()),tol=1e-12)
        sol = torch.tensor(sol).view(1,-1)
    elif solver == "direct": 
#         sol = np.linalg.inv( np.array(jac.detach().cpu()) )@np.array(rhs.detach().cpu())
        sol = (torch.linalg.solve( jac.detach(), rhs.detach())).view(1,-1)
    elif solver == "ls":
        sol = (torch.linalg.lstsq(jac.detach().cpu(),rhs.detach().cpu(),driver='gelsd').solution).view(1,-1)
        # sol = (torch.linalg.lstsq(jac.detach(),rhs.detach()).solution).view(1,-1) # gpu/cpu, driver = 'gels', cannot solve singular
    print("solving Ax = b time taken: ", time.time()-start_time)
    return sol 


### Test $L^2$ projection

In [4]:
def test_l2_projection():

    def u_exact(x):
        return torch.cos(pi*x[:,0:1])*torch.cos( pi*x[:,1:2]) * torch.cos(pi*x[:,2:3])  
    def alpha(x): 
        return torch.ones(x.size(0),1).to(device)

    g_N = None 
    integration_weights, integration_points = PiecewiseGQ3D_weights_points(30, 3)
    err_l2_list = [] 
    for neuron_num in [10,20,40,80,160]: 
        my_model = model(3, neuron_num, 1, k = 1).to(device) 
        my_model = adjust_neuron_position(my_model.cpu(),3).to(device)
        sol = minimize_linear_layer_explicit_assemble(my_model,u_exact,integration_weights, integration_points)
        my_model.fc2.weight.data[0,:] = sol[:] 
        diff_sqrd = (my_model(integration_points).detach() - u_exact(integration_points))**2
        err_l2 = torch.sqrt(torch.sum(integration_weights * diff_sqrd)) 
        print(err_l2)
        err_l2_list.append(err_l2) 
    print(err_l2_list) 

# test_l2_projection() 



### Test Neumann problem 

In [5]:
def test_linear_neumann():

    def u_exact(x):
        return torch.cos(pi*x[:,0:1])*torch.cos( pi*x[:,1:2]) * torch.cos(pi*x[:,2:3])  
    def alpha(x): 
        return torch.ones(x.size(0),1).to(device)

    def u_exact_grad():
        d = 3 
        def grad_1(x):
            return - pi* torch.sin(pi*x[:,0:1])*torch.cos( pi*x[:,1:2]) * torch.cos(pi*x[:,2:3])   
        def grad_2(x):
            return - pi* torch.cos(pi*x[:,0:1])*torch.sin( pi*x[:,1:2]) * torch.cos(pi*x[:,2:3])  
        def grad_3(x):
            return - pi* torch.cos(pi*x[:,0:1])*torch.cos( pi*x[:,1:2]) * torch.sin(pi*x[:,2:3])   
        
        u_grad=[grad_1, grad_2,grad_3] 

        return u_grad

    def target(x):
        z = (  3 * (pi)**2 + 1)*torch.cos( pi*x[:,0:1])*torch.cos( pi*x[:,1:2] ) * torch.cos(pi*x[:,2:3]) 
        return z 

    g_N = None 
    
    def g_N(dim):
        u_grad = u_exact_grad() 
        bcs_N = []
        for i in range(dim):
            bcs_N.append((i, u_grad[i]))
        return bcs_N
    
    integration_weights, integration_points = PiecewiseGQ3D_weights_points(50, 3)
    weights_bd, pts_bd = Neumann_boundary_quadrature_points_weights(M = 999,d =3)   
    err_l2_list = [] 
    for neuron_num in [10,20,40,80]: 
        my_model = model(3, neuron_num, 1, k = 1).to(device) 
        my_model = adjust_neuron_position(my_model.cpu(),3).to(device) 
        sol = minimize_linear_layer_H1_explicit_assemble_efficient(my_model,alpha, target,  \
                            g_N, integration_weights, integration_points, w_bd = weights_bd, pts_bd = pts_bd, \
                            activation = 'relu',solver="direct" ,memory=2**29)
        my_model.fc2.weight.data[0,:] = sol[:] 
        diff_sqrd = (my_model(integration_points).detach() - u_exact(integration_points))**2
        err_l2 = torch.sqrt(torch.sum(integration_weights * diff_sqrd)) 
        print(err_l2)
        err_l2_list.append(err_l2) 
    print(err_l2_list) 


def test_linear_neumann2():

    def u_exact(x):
        return torch.sin(pi*x[:,0:1])*torch.sin( pi*x[:,1:2]) * torch.sin(pi*x[:,2:3])  
    def alpha(x): 
        return torch.ones(x.size(0),1).to(device)

    def u_exact_grad():
        d = 3 
        def grad_1(x):
            return  pi* torch.cos(pi*x[:,0:1])*torch.sin( pi*x[:,1:2]) * torch.sin(pi*x[:,2:3])   
        def grad_2(x):
            return pi* torch.sin(pi*x[:,0:1])*torch.cos( pi*x[:,1:2]) * torch.sin(pi*x[:,2:3])  
        def grad_3(x):
            return pi* torch.sin(pi*x[:,0:1])*torch.sin( pi*x[:,1:2]) * torch.cos(pi*x[:,2:3])   
        
        u_grad=[grad_1, grad_2,grad_3] 

        return u_grad
    def laplace_u_exact(x):
        return - 3*pi**2 * torch.sin(pi*x[:,0:1])*torch.sin( pi*x[:,1:2]) * torch.sin(pi*x[:,2:3])
    
    def target(x):
        return - laplace_u_exact(x) + u_exact(x) 
    
    def g_N(dim):
        u_grad = u_exact_grad() 
        bcs_N = []
        for i in range(dim):
            bcs_N.append((i, u_grad[i]))
        return bcs_N
    
    integration_weights, integration_points = PiecewiseGQ3D_weights_points(25, 3)
    weights_bd, pts_bd = Neumann_boundary_quadrature_points_weights(M = 999,d =3)   
    err_l2_list = [] 
    for neuron_num in [10,20,40,80]: 
        my_model = model(3, neuron_num, 1, k = 1).to(device) 
        my_model = adjust_neuron_position(my_model.cpu(),3).to(device) 
        sol = minimize_linear_layer_H1_explicit_assemble_efficient(my_model,alpha, target,  \
                            g_N, integration_weights, integration_points, w_bd = weights_bd, pts_bd = pts_bd, \
                            activation = 'relu',solver="direct" ,memory=2**29)
        my_model.fc2.weight.data[0,:] = sol[:] 
        diff_sqrd = (my_model(integration_points).detach() - u_exact(integration_points))**2
        err_l2 = torch.sqrt(torch.sum(integration_weights * diff_sqrd)) 
        print(err_l2)
        err_l2_list.append(err_l2) 
    print(err_l2_list) 

# print("test zero flux")
# # test_linear_neumann() # zero flux 
# print()

# print("test non-zero flux")
# test_linear_neumann2() # with non-zero flux 


In [6]:
def test_linear_neumann3(): 
    freq = 2
    sigma = 0.15 
    def gaussian(x):
        return torch.exp(-torch.sum( (x - 0.5)**2,dim=1,keepdim=True)/(2 *sigma**2) ) 
    def gaussian_grad_1(x):
        return  gaussian(x) * (- (x[:,0:1] - 0.5)/(sigma**2) ) 
    def gaussian_grad_2(x):
        return  gaussian(x) * (- (x[:,1:2] - 0.5)/(sigma**2) ) 
    def gaussian_grad_3(x):
        return  gaussian(x) * (- (x[:,2:3] - 0.5)/(sigma**2) ) 
    
    def u_exact(x):
        return gaussian(x) * torch.cos(2*pi*freq*x[:,0:1]) 
    def alpha(x): 
        return torch.ones(x.size(0),1).to(device)

    def u_grad_1(x):
        return  torch.cos(2*pi*freq*x[:,0:1]) *gaussian_grad_1(x) \
                - 2*pi*freq * torch.sin(2*pi*freq*x[:,0:1]) * gaussian(x) 
    def u_grad_2(x):
        return torch.cos(2*pi*freq*x[:,0:1]) * gaussian_grad_2(x)
    def u_grad_3(x):
        return  torch.cos(2*pi*freq*x[:,0:1]) * gaussian_grad_3(x)

    def u_exact_grad():
        d = 3 
        def u_grad_1(x):
            return  torch.cos(2*pi*freq*x[:,0:1]) *gaussian_grad_1(x) \
                    - 2*pi*freq * torch.sin(2*pi*freq*x[:,0:1]) * gaussian(x) 
        def u_grad_2(x):
            return torch.cos(2*pi*freq*x[:,0:1]) * gaussian_grad_2(x)
        def u_grad_3(x):
            return  torch.cos(2*pi*freq*x[:,0:1]) * gaussian_grad_3(x)

        u_grad=[u_grad_1, u_grad_2,u_grad_3] 
        return u_grad
    
    def laplace_u_exact(x):
        return - 2*pi*freq * torch.sin(2*pi*freq*x[:,0:1]) *gaussian_grad_1(x) \
                + torch.cos(2*pi*freq*x[:,0:1])*( gaussian(x) * ( ((x[:,0:1] - 0.5)/(sigma**2))**2 -1/(sigma**2))  ) \
                -( (2*pi*freq)**2 * torch.cos(2*pi*freq*x[:,0:1]) * gaussian(x) + (2*pi*freq)*torch.sin(2*pi*freq*x[:,0:1]) * gaussian_grad_1(x) ) \
                + torch.cos(2*pi*freq*x[:,0:1]) * (gaussian(x) * ( ((x[:,1:2] - 0.5)/(sigma**2))**2 -1/(sigma**2) )  ) \
                + torch.cos(2*pi*freq*x[:,0:1]) * ( gaussian(x) * ( ((x[:,2:3] - 0.5)/(sigma**2))**2 -1/(sigma**2) )   ) \

    def target(x):
        return - laplace_u_exact(x) + u_exact(x)**3  
    
    def g_N(dim):
        u_grad = u_exact_grad() 
        bcs_N = []
        for i in range(dim):
            bcs_N.append((i, u_grad[i]))
        return bcs_N
    
    
    integration_weights, integration_points = PiecewiseGQ3D_weights_points(50, 3)
    weights_bd, pts_bd = Neumann_boundary_quadrature_points_weights(M = 999,d =3)   
    err_l2_list = [] 
    for neuron_num in [160,320]: 
        my_model = model(3, neuron_num, 1, k = 1).to(device) 
        my_model = adjust_neuron_position(my_model.cpu(),3).to(device) 
        sol = minimize_linear_layer_H1_explicit_assemble_efficient(my_model,alpha, target,  \
                            g_N, integration_weights, integration_points, w_bd = weights_bd, pts_bd = pts_bd, \
                            activation = 'relu',solver="direct" ,memory=2**29)
        my_model.fc2.weight.data[0,:] = sol[:] 
        diff_sqrd = (my_model(integration_points).detach() - u_exact(integration_points))**2
        err_l2 = torch.sqrt(torch.sum(integration_weights * diff_sqrd)) 
        print(err_l2)
        err_l2_list.append(err_l2) 
    print(err_l2_list) 
# test_linear_neumann3()

In [7]:
### 

### Test Newton solver 


In [19]:
    
## define the nonlinearity 
def nonlinear(v):
    return torch.sinh(v)

def nonlinear_prime(v):
    return torch.cosh(v)

def minimize_linear_layer_newton_method(model,alpha,target,weights, integration_points,weights_bd, integration_points_bd, g_N,activation = 'relu', solver = 'direct',memory=2**29):
    """
    calls the following functions (dependency): 
    1. GQ_piecewise_2D
    input: the nn model containing parameter 
    1. define the loss function  
    2. take derivative to extract the linear system A
    3. call the cg solver in scipy to solve the linear system 
    output: sol. solution of Ax = b
    """
    start_time = time.time() 
    w = model.fc1.weight.data 
    b = model.fc1.bias.data 
    neuron_num = b.size(0) 
    M = integration_points.size(0)
    dim = integration_points.size(1) 
    coef_alpha = alpha(integration_points) # alpha  
    basis_value_col = F.relu(integration_points @ w.t()+ b)**(model.k) 
    weighted_basis_value_col = basis_value_col * weights 
    newton_iters = 20 

    total_size = neuron_num * M # memory, number of floating numbers 
    print('total size: {} {} = {}'.format(neuron_num,M,total_size))
    num_batch = total_size//memory + 1 # divide according to memory
    print("num batches: ",num_batch)
    batch_size = M//num_batch
    
    jac = torch.zeros(b.size(0),b.size(0)).to(device)
    jac_fixed_part = torch.zeros(b.size(0),b.size(0)).to(device)
    rhs = torch.zeros(b.size(0),1).to(device)
    rhs_gN = torch.zeros(b.size(0),1).to(device)
    
    # Stiffness matrix term in the jacobian && gradient term in rhs
    for j in range(0,M,batch_size): 
        end_index = j + batch_size
        if model.k == 1:  
            derivative_comm_part = torch.heaviside(integration_points[j:end_index] @ w.t()+ b, ZERO) 
            for d in range(dim): 
                basis_value_dxi_col = derivative_comm_part * w.t()[d:d+1,:]
                weighted_basis_value_dx_col = basis_value_dxi_col * weights[j:end_index] * coef_alpha[j:end_index] 
                jac_fixed_part += weighted_basis_value_dx_col.t() @ basis_value_dxi_col 
        else:
            derivative_comm_part = model.k * F.relu(integration_points[j:end_index] @ w.t()+ b)**(model.k-1)
            for d in range(dim):  
                basis_value_dxi_col = derivative_comm_part * w.t()[d:d+1,:]
                weighted_basis_value_dx_col = basis_value_dxi_col * weights[j:end_index] * coef_alpha[j:end_index] 
                jac_fixed_part += weighted_basis_value_dx_col.t() @ basis_value_dxi_col 
    jac[:,:] = jac_fixed_part[:,:]
    # neumann boundary condition 
    if g_N != None:
        size_pts_bd = int(integration_points_bd.size(0)/(2*dim))
        bcs_N = g_N(dim)
        for ii, g_ii in bcs_N:
            #Another for loop needed if we need to divide the integration points into batches 
            weighted_g_N = -g_ii(integration_points_bd[2*ii*size_pts_bd:(2*ii+1)*size_pts_bd,:])* weights_bd[2*ii*size_pts_bd:(2*ii+1)*size_pts_bd,:]
            basis_value_bd_col = F.relu(integration_points_bd[2*ii*size_pts_bd:(2*ii+1)*size_pts_bd,:] @ w.t()+ b)**(model.k)
            rhs_gN += basis_value_bd_col.t() @ weighted_g_N

            weighted_g_N = g_ii(integration_points_bd[(2*ii+1)*size_pts_bd:(2*ii+2)*size_pts_bd,:])* weights_bd[(2*ii+1)*size_pts_bd:(2*ii+2)*size_pts_bd,:]
            basis_value_bd_col = F.relu(integration_points_bd[(2*ii+1)*size_pts_bd:(2*ii+2)*size_pts_bd,:] @ w.t()+ b)**(model.k)
            rhs_gN += basis_value_bd_col.t() @ weighted_g_N
    
    for i in range(newton_iters): 
        print("newton iteration: ", i+1) 
        for j in range(0,M,batch_size): 
            end_index = j + batch_size
            basis_value_col = F.relu(integration_points[j:end_index] @ w.t()+ b)**(model.k) 
            weighted_basis_value_col = basis_value_col * weights[j:end_index] 
            coef_func = nonlinear_prime(model(integration_points[j:end_index]).detach()) # Nonlinearity dependent
            # mass matrix with variable coefficients  
            jac += weighted_basis_value_col.t() @ (coef_func * basis_value_col)
            # f- u^3 term 
            rhs += weighted_basis_value_col.t() @ (target(integration_points[j:end_index]) - nonlinear(model(integration_points[j:end_index]).detach()) )

        # Gradient term in rhs
        for j in range(0,M,batch_size): 
            end_index = j + batch_size
            if model.k == 1:  
                derivative_comm_part = torch.heaviside(integration_points[j:end_index] @ w.t()+ b, ZERO) 
                for d in range(dim): 
                    basis_value_dxi_col = derivative_comm_part * w.t()[d:d+1,:]
                    weighted_basis_value_dx_col = basis_value_dxi_col * weights[j:end_index] * coef_alpha[j:end_index] 
#                     jac += weighted_basis_value_dx_col.t() @ basis_value_dxi_col 
                    dmy_model_dxi = model.evaluate_derivative(integration_points[j:end_index],d+1).detach() # this can be further optimized 
                    rhs -= weighted_basis_value_dx_col.t() @ dmy_model_dxi
            else:
                derivative_comm_part = model.k * F.relu(integration_points[j:end_index] @ w.t()+ b)**(model.k-1)
                for d in range(dim):  

                    basis_value_dxi_col = derivative_comm_part * w.t()[d:d+1,:]
                    weighted_basis_value_dx_col = basis_value_dxi_col * weights[j:end_index] * coef_alpha[j:end_index] 
#                     jac += weighted_basis_value_dx_col.t() @ basis_value_dxi_col 
                    dmy_model_dxi = model.evaluate_derivative(integration_points[j:end_index],d+1).detach() # this can be further optimized 
                    rhs -= weighted_basis_value_dx_col.t() @ dmy_model_dxi

        rhs += rhs_gN
        
        # print("assembling the matrix time taken: ", time.time()-start_time) 
        start_time = time.time()    
        if solver == "cg": 
            sol, exit_code = linalg.cg(np.array(jac.detach().cpu()),np.array(rhs.detach().cpu()),tol=1e-12)
            sol = torch.tensor(sol).view(1,-1)
        elif solver == "direct": 
    #         sol = np.linalg.inv( np.array(jac.detach().cpu()) )@np.array(rhs.detach().cpu())
            sol = (torch.linalg.solve( jac.detach(), rhs.detach())).view(1,-1)
        elif solver == "ls":
            sol = (torch.linalg.lstsq(jac.detach().cpu(),rhs.detach().cpu(),driver='gelsd').solution).view(1,-1)
            # sol = (torch.linalg.lstsq(jac.detach(),rhs.detach()).solution).view(1,-1) # gpu/cpu, driver = 'gels', cannot solve singular
        # print("solving Ax = b time taken: ", time.time()-start_time)
        ## update the solution 
        model.fc2.weight.data[0,:] += sol[0,:]
        
        # print("newton iteration: ", i) 
        sol_update_l2_norm = torch.norm(sol)
        nn_linear_layer_l2_norm = torch.norm(model.fc2.weight.data[0,:])
        residual_l2_norm = torch.norm(rhs) 
        # print("sol_update_l2_norm:{} \t residual l2 norm: {} ".format(sol_update_l2_norm, residual_l2_norm))
        tol = 1e-10
        print("sol_update_l2_norm:{} \t residual l2 norm: {} ".format(sol_update_l2_norm, residual_l2_norm))
        
        jac[:,:] = jac_fixed_part[:,:] 
        rhs[:,0] = 0

        if sol_update_l2_norm < tol*nn_linear_layer_l2_norm or sol_update_l2_norm < tol or residual_l2_norm < tol*1e-1: 
            print("converged at iteration: ", i+1 )
            print("sol_update_l2_norm:{} \t residual l2 norm: {} ".format(sol_update_l2_norm, residual_l2_norm))
            return model.fc2.weight.data[:,:] 
        
    print("Newton solver NOT converged at iteration!!! ")
    print("sol_update_l2_norm:{} \t residual l2 norm: {} ".format(sol_update_l2_norm, residual_l2_norm))

    return model.fc2.weight.data[:,:] 

In [18]:
def test_nonlinear_cubic():
    freq = 2 

    def u_exact(x):
        return torch.sin(pi*x[:,0:1])*torch.sin( pi*x[:,1:2]) * torch.sin(pi*x[:,2:3])  
    def alpha(x): 
        return torch.ones(x.size(0),1).to(device)

    def u_exact_grad():
        d = 3 
        def grad_1(x):
            return  pi* torch.cos(pi*x[:,0:1])*torch.sin( pi*x[:,1:2]) * torch.sin(pi*x[:,2:3])   
        def grad_2(x):
            return pi* torch.sin(pi*x[:,0:1])*torch.cos( pi*x[:,1:2]) * torch.sin(pi*x[:,2:3])  
        def grad_3(x):
            return pi* torch.sin(pi*x[:,0:1])*torch.sin( pi*x[:,1:2]) * torch.cos(pi*x[:,2:3])   
        
        u_grad=[grad_1, grad_2,grad_3] 

        return u_grad
    def laplace_u_exact(x):
        return - 3*pi**2 * torch.sin(pi*x[:,0:1])*torch.sin( pi*x[:,1:2]) * torch.sin(pi*x[:,2:3])
    
    def target(x):
        return - laplace_u_exact(x) + nonlinear(u_exact(x)) 
    
    def g_N(dim):
        u_grad = u_exact_grad() 
        bcs_N = []
        for i in range(dim):
            bcs_N.append((i, u_grad[i]))
        return bcs_N
    
    def u_exact_approx(x):
        return 0.7 * u_exact(x)

    def rhs(x):
        return  -laplace_u_exact(x) + nonlinear(u_exact(x)) 

    
    err_l2_list = [] 
    err_h10_list = []  
    weights, integration_points = PiecewiseGQ3D_weights_points(40, order = 3) 
    weights_bd, integration_points_bd = Neumann_boundary_quadrature_points_weights(999, d = 3) 
#     weights_bd, integration_points_bd = None, None   
    for neuron_num in [16,32,64,128,256]: 
        my_model = model(3, neuron_num, 1, k = 3).to(device) 
        my_model = adjust_neuron_position(my_model.cpu(),3).to(device)
        sol = minimize_linear_layer_explicit_assemble(my_model,u_exact_approx,weights, integration_points,solver="direct")
        # sol = minimize_linear_layer_neumann(my_model,rhs_neumann,weights, integration_points,activation = 'relu', solver = 'direct')
        my_model.fc2.weight.data[0,:] = sol[0,:]   
        sol = minimize_linear_layer_newton_method(my_model,alpha,rhs, \
                    weights, integration_points,weights_bd, integration_points_bd,\
                     g_N,activation = 'relu',solver="direct",memory=2**29) 
        my_model.fc2.weight.data[0,:] = sol[0,:]
        # plot_2D(my_model)
        diff_sqrd = (my_model(integration_points).detach() - u_exact(integration_points))**2
        err_l2 = (weights.t() @ diff_sqrd)**0.5 
        err_l2_list.append(err_l2)
    print(err_l2_list)   
    return 0 
    
test_nonlinear_cubic()

assembling the matrix time taken:  0.00035572052001953125
solving Ax = b time taken:  0.0019559860229492188
total size: 16 1728000 = 27648000
num batches:  1
newton iteration:  1
sol_update_l2_norm:154504.80790412513 	 residual l2 norm: 0.14961280827288506 
newton iteration:  2
sol_update_l2_norm:1.4411919917686873 	 residual l2 norm: 0.00023887759256700398 
newton iteration:  3
sol_update_l2_norm:2.697788426533029e-06 	 residual l2 norm: 4.676468689361084e-10 
newton iteration:  4
sol_update_l2_norm:9.695976516407447e-12 	 residual l2 norm: 1.2578361866822073e-15 
converged at iteration:  4
sol_update_l2_norm:9.695976516407447e-12 	 residual l2 norm: 1.2578361866822073e-15 
assembling the matrix time taken:  0.00025463104248046875
solving Ax = b time taken:  0.003767251968383789
total size: 32 1728000 = 55296000
num batches:  1
newton iteration:  1
sol_update_l2_norm:11977.98123713716 	 residual l2 norm: 0.21064818628149498 
newton iteration:  2
sol_update_l2_norm:1.6228859664869282 	

0

#### Test Gabor function 


In [10]:
def test2_nonlinear_cubic():
    freq = 3
    sigma = 0.15 
    def gaussian(x):
        return torch.exp(-torch.sum( (x - 0.5)**2,dim=1,keepdim=True)/(2 *sigma**2) ) 
    def gaussian_grad_1(x):
        return  gaussian(x) * (- (x[:,0:1] - 0.5)/(sigma**2) ) 
    def gaussian_grad_2(x):
        return  gaussian(x) * (- (x[:,1:2] - 0.5)/(sigma**2) ) 
    def gaussian_grad_3(x):
        return  gaussian(x) * (- (x[:,2:3] - 0.5)/(sigma**2) ) 
    
    def u_exact(x):
        return gaussian(x) * torch.cos(2*pi*freq*x[:,0:1]) 
    def alpha(x): 
        return torch.ones(x.size(0),1).to(device)

    def u_grad_1(x):
        return  torch.cos(2*pi*freq*x[:,0:1]) *gaussian_grad_1(x) \
                - 2*pi*freq * torch.sin(2*pi*freq*x[:,0:1]) * gaussian(x) 
    def u_grad_2(x):
        return torch.cos(2*pi*freq*x[:,0:1]) * gaussian_grad_2(x)
    def u_grad_3(x):
        return  torch.cos(2*pi*freq*x[:,0:1]) * gaussian_grad_3(x)

    def u_exact_grad():
        d = 3 
        def u_grad_1(x):
            return  torch.cos(2*pi*freq*x[:,0:1]) *gaussian_grad_1(x) \
                    - 2*pi*freq * torch.sin(2*pi*freq*x[:,0:1]) * gaussian(x) 
        def u_grad_2(x):
            return torch.cos(2*pi*freq*x[:,0:1]) * gaussian_grad_2(x)
        def u_grad_3(x):
            return  torch.cos(2*pi*freq*x[:,0:1]) * gaussian_grad_3(x)

        u_grad=[u_grad_1, u_grad_2,u_grad_3] 
        return u_grad
    
    def laplace_u_exact(x):
        return - 2*pi*freq * torch.sin(2*pi*freq*x[:,0:1]) *gaussian_grad_1(x) \
                + torch.cos(2*pi*freq*x[:,0:1])*( gaussian(x) * ( ((x[:,0:1] - 0.5)/(sigma**2))**2 -1/(sigma**2))  ) \
                -( (2*pi*freq)**2 * torch.cos(2*pi*freq*x[:,0:1]) * gaussian(x) + (2*pi*freq)*torch.sin(2*pi*freq*x[:,0:1]) * gaussian_grad_1(x) ) \
                + torch.cos(2*pi*freq*x[:,0:1]) * (gaussian(x) * ( ((x[:,1:2] - 0.5)/(sigma**2))**2 -1/(sigma**2) )  ) \
                + torch.cos(2*pi*freq*x[:,0:1]) * ( gaussian(x) * ( ((x[:,2:3] - 0.5)/(sigma**2))**2 -1/(sigma**2) )   ) \

    def target(x):
        return - laplace_u_exact(x) + nonlinear(u_exact(x)) 
    
    def g_N(dim):
        u_grad = u_exact_grad() 
        bcs_N = []
        for i in range(dim):
            bcs_N.append((i, u_grad[i]))
        return bcs_N
    
    def u_exact_approx(x):
        return 0.99 * u_exact(x)

    def rhs(x):
        return  -laplace_u_exact(x) + nonlinear(u_exact(x)) 

    err_l2_list = [] 
    err_h10_list = []  
    weights, integration_points = PiecewiseGQ3D_weights_points(50, order = 3) 
    weights_bd, integration_points_bd = Neumann_boundary_quadrature_points_weights(999, d = 3) 
#     weights_bd, integration_points_bd = None, None   
    for neuron_num in [64,128]: 
        my_model = model(3, neuron_num, 1, k = 3).to(device) 
        my_model = adjust_neuron_position(my_model.cpu(),3).to(device)
        sol = minimize_linear_layer_explicit_assemble(my_model,u_exact_approx,weights, integration_points,solver="direct")
        # sol = minimize_linear_layer_neumann(my_model,rhs_neumann,weights, integration_points,activation = 'relu', solver = 'direct')
        my_model.fc2.weight.data[0,:] = sol[0,:]   
        sol = minimize_linear_layer_newton_method(my_model,alpha,rhs, \
                    weights, integration_points,weights_bd, integration_points_bd,\
                     g_N,activation = 'relu',solver="direct",memory=2**29) 
        my_model.fc2.weight.data[0,:] = sol[0,:]
        # plot_2D(my_model)
        diff_sqrd = (my_model(integration_points).detach() - u_exact(integration_points))**2
        err_l2 = (weights.t() @ diff_sqrd)**0.5 
        err_l2_list.append(err_l2)
    print(err_l2_list)   
    return 0 

# test2_nonlinear_cubic()

## compare autograd and manually computing derivatives 

In [11]:
import torch

# Constants
freq = 2
sigma = 0.15
pi = torch.pi
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Gaussian function and its gradients
def gaussian(x):
    return torch.exp(-torch.sum((x - 0.5)**2, dim=1, keepdim=True) / (2 * sigma**2))

def gaussian_grad_1(x):
    return gaussian(x) * (-(x[:, 0:1] - 0.5) / (sigma**2))

def gaussian_grad_2(x):
    return gaussian(x) * (-(x[:, 1:2] - 0.5) / (sigma**2))

def gaussian_grad_3(x):
    return gaussian(x) * (-(x[:, 2:3] - 0.5) / (sigma**2))

# Exact function u_exact
def u_exact(x):
    return gaussian(x) * torch.cos(2 * pi * freq * x[:, 0:1])

# Manually computed gradients
def u_grad_1(x):
    return torch.cos(2 * pi * freq * x[:, 0:1]) * gaussian_grad_1(x) - 2 * pi * freq * torch.sin(2 * pi * freq * x[:, 0:1]) * gaussian(x)

def u_grad_2(x):
    return torch.cos(2 * pi * freq * x[:, 0:1]) * gaussian_grad_2(x)

def u_grad_3(x):
    return torch.cos(2 * pi * freq * x[:, 0:1]) * gaussian_grad_3(x)

# Collecting the gradients in a list
def u_exact_grad():
    return [u_grad_1, u_grad_2, u_grad_3]

# # Manually computed Laplacian
def laplace_u_exact(x):
    return - 2*pi*freq * torch.sin(2*pi*freq*x[:,0:1]) *gaussian_grad_1(x) \
            + torch.cos(2*pi*freq*x[:,0:1])*( gaussian(x) * ( ((x[:,0:1] - 0.5)/(sigma**2))**2 -1/(sigma**2))  ) \
            -( (2*pi*freq)**2 * torch.cos(2*pi*freq*x[:,0:1]) * gaussian(x) + (2*pi*freq)*torch.sin(2*pi*freq*x[:,0:1]) * gaussian_grad_1(x) ) \
            + torch.cos(2*pi*freq*x[:,0:1]) * (gaussian(x) * ( ((x[:,1:2] - 0.5)/(sigma**2))**2 -1/(sigma**2) )  ) \
            + torch.cos(2*pi*freq*x[:,0:1]) * ( gaussian(x) * ( ((x[:,2:3] - 0.5)/(sigma**2))**2 -1/(sigma**2) )   ) \


# def laplace_u_exact(x):
#     # Second derivative w.r.t x_1
#     term_1 = -(2 * pi * freq)**2 * torch.cos(2 * pi * freq * x[:, 0:1]) * gaussian(x)
#     term_1 += torch.cos(2 * pi * freq * x[:, 0:1]) * gaussian(x) * ((x[:, 0:1] - 0.5)**2 / sigma**4 - 1 / sigma**2)
#     term_1 += 2 * pi * freq * torch.sin(2 * pi * freq * x[:, 0:1]) * gaussian_grad_1(x)

#     # Second derivative w.r.t x_2
#     term_2 = torch.cos(2 * pi * freq * x[:, 0:1]) * gaussian(x) * ((x[:, 1:2] - 0.5)**2 / sigma**4 - 1 / sigma**2)

#     # Second derivative w.r.t x_3
#     term_3 = torch.cos(2 * pi * freq * x[:, 0:1]) * gaussian(x) * ((x[:, 2:3] - 0.5)**2 / sigma**4 - 1 / sigma**2)

#     return term_1 + term_2 + term_3


# Function to compute gradient using autograd
def compute_autograd_grad(u_func, x):
    x.requires_grad_(True)  # Enable gradient tracking for input x
    u = u_func(x)
    u_grad = torch.autograd.grad(outputs=u, inputs=x,
                                 grad_outputs=torch.ones_like(u),
                                 create_graph=True, retain_graph=True)[0]
    return u_grad

# Function to compute Laplacian using autograd
def compute_autograd_laplace(u_func, x):
    u_grad = compute_autograd_grad(u_func, x)
    
    laplacian = 0
    for i in range(x.shape[1]):
        grad_i = u_grad[:, i:i+1]
        u_grad2_i = torch.autograd.grad(outputs=grad_i, inputs=x,
                                        grad_outputs=torch.ones_like(grad_i),
                                        create_graph=True, retain_graph=True)[0][:, i:i+1]
        laplacian += u_grad2_i
    return laplacian

# Generate sample input points
x = torch.rand(1000, 3).to(device)  # Random 3D points

# Compute the manually computed gradients and Laplacian
s_time = time.time()
u_grad_manual_1 = u_grad_1(x)
u_grad_manual_2 = u_grad_2(x)
u_grad_manual_3 = u_grad_3(x)
laplace_manual = laplace_u_exact(x)
print("manully compute derivative: ",time.time() - s_time)

# Compute the gradients and Laplacian using autograd
s_time = time.time()
u_grad_autograd = compute_autograd_grad(u_exact, x)
laplace_autograd = compute_autograd_laplace(u_exact, x)
print("autograd for derivative: ",time.time() - s_time)
# Compute differences for comparison
grad_diff_1 = torch.abs(u_grad_manual_1 - u_grad_autograd[:, 0:1])
grad_diff_2 = torch.abs(u_grad_manual_2 - u_grad_autograd[:, 1:2])
grad_diff_3 = torch.abs(u_grad_manual_3 - u_grad_autograd[:, 2:3])
laplace_diff = torch.abs(laplace_manual - laplace_autograd)

# Display the results
print("Manual Gradient 1 vs Autograd Gradient 1 Difference:")
print(grad_diff_1.sum())

print("\nManual Gradient 2 vs Autograd Gradient 2 Difference:")
print(grad_diff_2.sum())

print("\nManual Gradient 3 vs Autograd Gradient 3 Difference:")
print(grad_diff_3.sum())

print("\nManual Laplacian vs Autograd Laplacian Difference:")
print(laplace_diff.sum())


manully compute derivative:  0.0022568702697753906
autograd for derivative:  0.0037720203399658203
Manual Gradient 1 vs Autograd Gradient 1 Difference:
tensor(3.5875e-14, device='cuda:0', grad_fn=<SumBackward0>)

Manual Gradient 2 vs Autograd Gradient 2 Difference:
tensor(1.6071e-14, device='cuda:0', grad_fn=<SumBackward0>)

Manual Gradient 3 vs Autograd Gradient 3 Difference:
tensor(1.4882e-14, device='cuda:0', grad_fn=<SumBackward0>)

Manual Laplacian vs Autograd Laplacian Difference:
tensor(1.0734e-12, device='cuda:0', grad_fn=<SumBackward0>)


## CGA Nonlinear Problem 

In [20]:
def select_greedy_neuron_ind(relu_dict_parameters,my_model,target,gw_expand, integration_points,g_N,weights_bd, integration_points_bd,k,memory = 2**29):
    dim = integration_points.size(1) 
    M = integration_points.size(0)
    N0 = relu_dict_parameters.size(0)   
    neuron_num = my_model.fc2.weight.size(1) if my_model != None else 0

    output = torch.zeros(N0,1).to(device) 
    s_time = time.time()
    total_size2 = M*(neuron_num+1)
    num_batch2 = total_size2//memory + 1 
    batch_size_2 = M//num_batch2 # integration points 
    # N(u) - f terms, divide the integration points into batches 
    if my_model != None: 
        func_values = - target(integration_points) 
        for jj in range(0,M,batch_size_2): 
            end_index = jj + batch_size_2 
            model_values = nonlinear(my_model(integration_points[jj:end_index,:]).detach()) 
            func_values[jj:end_index,:] += model_values #Change 1.  
    else: 
        func_values = - target(integration_points)    
    weight_func_values = func_values*gw_expand  
    
    total_size = M * N0 
    num_batch = total_size//memory + 1 
    batch_size_1 = N0//num_batch # dictionary elements
    print("======argmax subproblem:f and N(u) terms, num batches: ",num_batch)
    for j in range(0,N0,batch_size_1):
        end_index = j + batch_size_1 
        basis_values = (F.relu( torch.matmul(integration_points,relu_dict_parameters[j:end_index,0:dim].T ) - relu_dict_parameters[j:end_index,dim])**k).T # uses broadcasting
        output[j:end_index] += torch.matmul(basis_values,weight_func_values) #
    print('======TIME=======f and N(u) terms time :',time.time()-s_time)
    
    # Gradient term: <\nabla u_n, \nabla g_i>, i = 1,2,3,...,N
    ## ============================================================================
    s_time =time.time() 
    if my_model!= None:
        #compute the derivative of the model 
        model_derivative_values = torch.zeros(M,dim).to(device) 
        for d in range(dim): ## there is a more efficient way 
            for jj in range(0,M,batch_size_2):
                end_index = jj + batch_size_2 
                model_derivative_values[jj:end_index,d:d+1] = my_model.evaluate_derivative(integration_points[jj:end_index,:],d+1).detach()
                
        if my_model.k == 1: 
            #compute the derivative of the dictionary elements 
            for j in range(0,N0,batch_size_1): 
                end_index = j + batch_size_1 
                weighted_derivative_part = gw_expand * torch.heaviside(integration_points@ (relu_dict_parameters[j:end_index,0:dim].T) - relu_dict_parameters[j:end_index,dim], ZERO)
                for d in range(dim):
                    weighted_basis_value_dx_col = weighted_derivative_part * relu_dict_parameters.t()[d:d+1,j:end_index] 
                    output[j:end_index] += weighted_basis_value_dx_col.t() @ model_derivative_values[:,d:d+1]
        else:
            #compute the derivative of the dictionary elements 
            for j in range(0,N0,batch_size_1):  
                end_index = j + batch_size_1
                weighted_derivative_part = gw_expand *my_model.k * F.relu(integration_points@ (relu_dict_parameters[j:end_index,0:dim].T) - relu_dict_parameters[j:end_index,dim])**(my_model.k-1)
                for d in range(dim):
                    weighted_basis_value_dx_col = weighted_derivative_part * relu_dict_parameters.t()[d:d+1,j:end_index]
                    output[j:end_index] += weighted_basis_value_dx_col.t() @ model_derivative_values[:,d:d+1]

    print('======TIME=======stiffness matrix terms time :',time.time()-s_time)
    
    #Neumann boundary condition
    s_time =time.time()  
    output4 = 0 
    if g_N != None:
        size_pts_bd = int(integration_points_bd.size(0)/(2*dim)) # pre-defined rules for integration points on bdries
        bcs_N = g_N(dim)
        for ii, g_ii in bcs_N:
            # pts_bd_ii = pts_bd[2*ii*size_pts_bd:(2*ii+1)*size_pts_bd,:]
            weighted_g_N = -g_ii(integration_points_bd[2*ii*size_pts_bd:(2*ii+1)*size_pts_bd,:])* weights_bd[2*ii*size_pts_bd:(2*ii+1)*size_pts_bd,:]
            basis_value_bd_col = F.relu(integration_points_bd[2*ii*size_pts_bd:(2*ii+1)*size_pts_bd,:] @ (relu_dict_parameters[:,0:dim].T) - relu_dict_parameters[:,dim] )**(k)
            output4 += basis_value_bd_col.t() @ weighted_g_N

            weighted_g_N = g_ii(integration_points_bd[(2*ii+1)*size_pts_bd:(2*ii+2)*size_pts_bd,:])* weights_bd[(2*ii+1)*size_pts_bd:(2*ii+2)*size_pts_bd,:]
            basis_value_bd_col = F.relu(integration_points_bd[(2*ii+1)*size_pts_bd:(2*ii+2)*size_pts_bd,:] @ (relu_dict_parameters[:,0:dim].T) - relu_dict_parameters[:,dim])**(k)
            output4 += basis_value_bd_col.t() @ weighted_g_N
    print('======TIME=======neumann bd terms time :',time.time()-s_time) 
    output -= output4
    output = torch.abs(output) 
    
    neuron_index = torch.argmax(output.flatten())
    return neuron_index 

def L2_projection_init(model,sol,weights,integration_points,activation = 'relu', solver = 'direct'):
    start_time = time.time() 
    w = model.fc1.weight.data 
    b = model.fc1.bias.data 
    basis_value_col = F.relu(integration_points @ w.t()+ b)**(model.k) 
    weighted_basis_value_col = basis_value_col * weights 
    jac = weighted_basis_value_col.t() @ basis_value_col 
      
    rhs = jac[:,:-1] @ sol.t()

    print("assembling the matrix time taken: ", time.time()-start_time) 
    start_time = time.time()    
    if solver == "cg": 
        sol, exit_code = linalg.cg(np.array(jac.detach().cpu()),np.array(rhs.detach().cpu()),tol=1e-12)
        sol = torch.tensor(sol).view(1,-1)
    elif solver == "direct": 
#         sol = np.linalg.inv( np.array(jac.detach().cpu()) )@np.array(rhs.detach().cpu())
        sol = (torch.linalg.solve( jac.detach(), rhs.detach())).view(1,-1)
    elif solver == "ls":
        sol = (torch.linalg.lstsq(jac.detach().cpu(),rhs.detach().cpu(),driver='gelsd').solution).view(1,-1)
        # sol = (torch.linalg.lstsq(jac.detach(),rhs.detach()).solution).view(1,-1) # gpu/cpu, driver = 'gels', cannot solve singular
    print("solving Ax = b time taken: ", time.time()-start_time)
    model.fc2.weight.data[0,:] = sol[0,:]  
    return model 

def CGANonlinearPoissonReLU3D(my_model,target,alpha,u_exact, u_exact_grad,g_N, N_list,num_epochs,plot_freq, Nx, order, k =1, rand_deter = 'deter', linear_solver = "direct",memory = 2**29): 
    """ Orthogonal greedy algorithm using 1D ReLU dictionary over [-pi,pi]
    Parameters
    ----------
    my_model: 
        nn model 
    target: 
        rhs hand side function for a PDE 
    u_exact:
        exact solution 
    u_exact_grad:
        a function that returns gradient of the exact solution in a list 
    g_N: 
        a function that returns gradient of the exact solution with numbers  
    num_epochs: int 
        number of training epochs 
    integration_intervals: int 
        number of subintervals for piecewise numerical quadrature 

    Returns
    -------
    err: tensor 
        rank 1 torch tensor to record the L2 error history  
    model: 
        trained nn model 
    """
    gw_expand, integration_points = PiecewiseGQ3D_weights_points(Nx, order)
    dim = integration_points.size(1) 
    M = integration_points.size(0)
    weights_bd, integration_points_bd = Neumann_boundary_quadrature_points_weights(99999999,dim) 

    # Compute initial L2 error and the gradient error 
    err = torch.zeros(num_epochs+1).to(device)
    err_h10 = torch.zeros(num_epochs+1).to(device)
    num_neuron = 0 if my_model == None else int(my_model.fc1.bias.detach().data.size(0))
    total_size2 = M*(num_neuron+1)
    num_batch2 = total_size2//memory + 1 
    batch_size_2 = M//num_batch2 # integration points 
    if my_model == None: 
        for jj in range(0,M,batch_size_2): 
            end_index = jj + batch_size_2 
            func_values = target(integration_points[jj:end_index,:])
            err[0] += torch.sum(func_values**2 * gw_expand[jj:end_index,:])**0.5
        list_b = []
        list_w = []
    else: 
        bias = my_model.fc1.bias.detach().data
        weights = my_model.fc1.weight.detach().data
        for jj in range(0,M,batch_size_2): 
            end_index = jj + batch_size_2 
            func_values = u_exact(integration_points[jj:end_index,:]) - my_model(integration_points[jj:end_index,:]).detach()
            err[0] += torch.sum(func_values**2 * gw_expand[jj:end_index,:])**0.5
        list_b = list(bias)
        list_w = list(weights)
        sol = my_model.fc2.weight.data[0,:]

    # initial gradient error 
    if u_exact_grad != None and my_model!=None:
        u_grad = u_exact_grad() 
        for ii, grad_i in enumerate(u_grad): 
            for jj in range(0,M,batch_size_2): 
                end_index = jj + batch_size_2 
                my_model_dxi = my_model.evaluate_derivative(integration_points[jj:end_index,:],ii+1).detach() 
                err_h10[0] += torch.sum((grad_i(integration_points[jj:end_index,:]) - my_model_dxi)**2 * gw_expand[jj:end_index,:])**0.5
    elif u_exact_grad != None and my_model==None:
        u_grad = u_exact_grad() 
        for grad_i in u_grad: 
            for jj in range(0,M,batch_size_2): 
                end_index = jj + batch_size_2 
                err_h10[0] += torch.sum((grad_i(integration_points[jj:end_index,:]))**2 * gw_expand[jj:end_index,:])**0.5
    
    start_time = time.time()
    solver = linear_solver
    N0 = np.prod(N_list)
    if rand_deter == 'deter':
        relu_dict_parameters = generate_relu_dict3D(N_list).to(device)
    print("using linear solver: ",solver)
    # CGA training loop 
    for i in range(num_epochs): 
        print("epoch: ",i+1, end = '\t')
        if rand_deter == 'rand':
            relu_dict_parameters = generate_relu_dict3D_QMC(1,N0).to(device) 
        
        time_argmax = time.time()
        neuron_index = select_greedy_neuron_ind(relu_dict_parameters,my_model,target,gw_expand, integration_points,g_N,weights_bd, integration_points_bd,k,memory=memory)
        print("=======> argmax subproblem time: ",time.time() - time_argmax)
        # print(neuron_index)
        list_w.append(relu_dict_parameters[neuron_index,0:dim]) # 
        list_b.append(-relu_dict_parameters[neuron_index,dim])
        num_neuron += 1
        my_model = model(dim,num_neuron,1,k).to(device)
        w_tensor = torch.stack(list_w, 0 ) 
        b_tensor = torch.tensor(list_b)
        my_model.fc1.weight.data[:,:] = w_tensor[:,:]
        my_model.fc1.bias.data[:] = b_tensor[:]

        ##todo 
        if num_neuron <=2: 
            my_model.fc2.weight.data[0,:] = 0.0001
        else: 
            ## L2 projection onto previous solution as the initial guess 
            my_model.fc2.weight.data[0,:num_neuron -1 ] = sol[:] # projection of previous solution
            my_model = L2_projection_init(my_model,sol,gw_expand,integration_points,activation = 'relu', solver = solver) 

        sol = minimize_linear_layer_newton_method(my_model,alpha, target,\
                    gw_expand, integration_points,weights_bd, integration_points_bd,\
                    g_N,activation ='relu', solver = solver)
        
        sol = sol.flatten() 
        my_model.fc2.weight.data[0,:] = sol[:]

        # Get L2 error and gradient error 
        total_size2 = M*(num_neuron+1)
        num_batch2 = total_size2//memory + 1 
        batch_size_2 = M//num_batch2 # integration points 
        
        for jj in range(0,M,batch_size_2):
            end_index = jj + batch_size_2 
            func_values = u_exact(integration_points[jj:end_index,:]) - my_model(integration_points[jj:end_index,:]).detach()
            func_values = func_values**2 
            err[i+1]+= torch.sum(func_values*gw_expand[jj:end_index,:])**0.5

        if u_exact_grad != None:
            for ii, grad_i in enumerate(u_grad): 
                for jj in range(0,M,batch_size_2): 
                    end_index = jj + batch_size_2 
                    my_model_dxi = my_model.evaluate_derivative(integration_points[jj:end_index,:],ii+1).detach() 
                    err_h10[i+1] += torch.sum((grad_i(integration_points[jj:end_index,:]) - my_model_dxi)**2 * gw_expand[jj:end_index,:])**0.5

    print("time taken: ",time.time() - start_time)
    return err.cpu(), err_h10.cpu(), my_model


In [21]:
freq = 1
def u_exact(x):
    return torch.cos(freq*pi*x[:,0:1])*torch.cos( freq*pi*x[:,1:2]) * torch.cos(freq*pi*x[:,2:3])  
def alpha(x): 
    return torch.ones(x.size(0),1).to(device)

def u_exact_grad():
    d = 3 
    def grad_1(x):
        return - freq*pi* torch.sin(freq*pi*x[:,0:1])*torch.cos( freq*pi*x[:,1:2]) * torch.cos(freq*pi*x[:,2:3])   
    def grad_2(x):
        return - freq*pi* torch.cos(freq*pi*x[:,0:1])*torch.sin( freq*pi*x[:,1:2]) * torch.cos(freq*pi*x[:,2:3])  
    def grad_3(x):
        return - freq*pi* torch.cos(freq*pi*x[:,0:1])*torch.cos( freq*pi*x[:,1:2]) * torch.sin(freq*pi*x[:,2:3])   
    
    u_grad=[grad_1, grad_2,grad_3] 

    return u_grad
def laplace_u_exact(x):
    return -3*(freq*pi)**2 * torch.cos(freq*pi*x[:,0:1])*torch.cos( freq*pi*x[:,1:2]) * torch.cos(freq*pi*x[:,2:3])
# def target(x):
#     z = -laplace_u_exact(x) + u_exact(x)**3 
#     return z 

def u_exact_approx(x):
    return 0.7 * u_exact(x)

def rhs(x):
    return  -laplace_u_exact(x) + nonlinear(u_exact(x))

g_N = None 

function_name = "cos4pix" 
filename_write = "3DCGA-{}-order.txt".format(function_name)
f_write = open(filename_write, "a")
f_write.write("\n")
f_write.close() 
save = False 
relu_k = 3
for N_list in [[2**3,2**3,2**3]]: # ,[2**6,2**6],[2**7,2**7] 
    # save = True 
    f_write = open(filename_write, "a")
    my_model = None 
    Nx = 100
    order = 3
    exponent = 5
    num_epochs = 2**exponent  
    plot_freq = num_epochs 
    N = np.prod(N_list)
    err_QMC2, err_h10, my_model = CGANonlinearPoissonReLU3D(my_model,rhs,alpha, u_exact, u_exact_grad,g_N, N_list,num_epochs,plot_freq, Nx, order, k = relu_k, rand_deter = 'rand', linear_solver = "direct")
    if save: 
        folder = 'data-neumann/'
        filename = folder + 'err_OGA_2D_{}_neuron_{}_N_{}_deterministic.pt'.format(function_name,num_epochs,N)
        torch.save(err_QMC2,filename) 
        folder = 'data-neumann/'
        filename = folder + 'model_OGA_2D_{}_neuron_{}_N_{}_deterministic.pt'.format(function_name,num_epochs,N)
        torch.save(my_model,filename)

    show_convergence_order(err_QMC2,err_h10,exponent,N,filename_write,False)
    show_convergence_order_latex(err_QMC2,err_h10,exponent,k =relu_k,d = 3)

using linear solver:  direct
epoch:  1	======argmax subproblem:f and N(u) terms, num batches:  26
======TIME=======f and N(u) terms time : 0.00968623161315918
======TIME=======stiffness matrix terms time : 1.6689300537109375e-06
======TIME=======neumann bd terms time : 9.298324584960938e-06
=======> argmax subproblem time:  0.009844064712524414
total size: 1 27000000 = 27000000
num batches:  1
newton iteration:  1
sol_update_l2_norm:0.01078851561903999 	 residual l2 norm: 0.2939186666048791 
newton iteration:  2
sol_update_l2_norm:5.375800649556315e-07 	 residual l2 norm: 1.4647860005823077e-05 
newton iteration:  3
sol_update_l2_norm:4.106123599433748e-15 	 residual l2 norm: 1.1188272530660015e-13 
converged at iteration:  3
sol_update_l2_norm:4.106123599433748e-15 	 residual l2 norm: 1.1188272530660015e-13 
epoch:  2	======argmax subproblem:f and N(u) terms, num batches:  26
======TIME=======f and N(u) terms time : 0.009660482406616211
======TIME=======stiffness matrix terms time : 1

======TIME=======stiffness matrix terms time : 1.1586072444915771
======TIME=======neumann bd terms time : 2.1457672119140625e-06
=======> argmax subproblem time:  1.1759252548217773
assembling the matrix time taken:  0.00042939186096191406
solving Ax = b time taken:  0.02275872230529785
total size: 10 27000000 = 270000000
num batches:  1
newton iteration:  1
sol_update_l2_norm:0.6856423719610536 	 residual l2 norm: 0.8654217504138438 
newton iteration:  2
sol_update_l2_norm:0.0009038680919743307 	 residual l2 norm: 0.012018091922618837 
newton iteration:  3
sol_update_l2_norm:1.9819741922646746e-08 	 residual l2 norm: 3.304949232477068e-07 
newton iteration:  4
sol_update_l2_norm:5.6361912502261825e-15 	 residual l2 norm: 7.203592748597016e-14 
converged at iteration:  4
sol_update_l2_norm:5.6361912502261825e-15 	 residual l2 norm: 7.203592748597016e-14 
epoch:  11	======argmax subproblem:f and N(u) terms, num batches:  26
======TIME=======f and N(u) terms time : 0.010068416595458984


======TIME=======stiffness matrix terms time : 1.2003979682922363
======TIME=======neumann bd terms time : 1.9073486328125e-06
=======> argmax subproblem time:  1.2177391052246094
assembling the matrix time taken:  0.00046181678771972656
solving Ax = b time taken:  0.03406929969787598
total size: 19 27000000 = 513000000
num batches:  1
newton iteration:  1
sol_update_l2_norm:0.9892556824692966 	 residual l2 norm: 0.016649582812112547 
newton iteration:  2
sol_update_l2_norm:1.9400007508736594e-05 	 residual l2 norm: 9.955551382601156e-05 
newton iteration:  3
sol_update_l2_norm:7.526343171068411e-13 	 residual l2 norm: 1.8031683024873528e-11 
converged at iteration:  3
sol_update_l2_norm:7.526343171068411e-13 	 residual l2 norm: 1.8031683024873528e-11 
epoch:  20	======argmax subproblem:f and N(u) terms, num batches:  26
======TIME=======f and N(u) terms time : 0.010226249694824219
======TIME=======stiffness matrix terms time : 1.2095553874969482
======TIME=======neumann bd terms time 

======TIME=======stiffness matrix terms time : 1.250641107559204
======TIME=======neumann bd terms time : 1.430511474609375e-06
=======> argmax subproblem time:  1.2684528827667236
assembling the matrix time taken:  0.0004878044128417969
solving Ax = b time taken:  0.043949127197265625
total size: 28 27000000 = 756000000
num batches:  2
newton iteration:  1
sol_update_l2_norm:7.766073054907539 	 residual l2 norm: 0.008705229450404015 
newton iteration:  2
sol_update_l2_norm:0.0005776627023410875 	 residual l2 norm: 0.0003619768959102803 
newton iteration:  3
sol_update_l2_norm:1.6782279924346238e-10 	 residual l2 norm: 3.2438479946347287e-10 
converged at iteration:  3
sol_update_l2_norm:1.6782279924346238e-10 	 residual l2 norm: 3.2438479946347287e-10 
epoch:  29	======argmax subproblem:f and N(u) terms, num batches:  26
======TIME=======f and N(u) terms time : 0.010272979736328125
======TIME=======stiffness matrix terms time : 1.2492146492004395
======TIME=======neumann bd terms time

## Gabor function 

In [14]:
## Gabor function test 
freq = 2
sigma = 0.15 
def gaussian(x):
    return torch.exp(-torch.sum( (x - 0.5)**2,dim=1,keepdim=True)/(2 *sigma**2) ) 
def gaussian_grad_1(x):
    return  gaussian(x) * (- (x[:,0:1] - 0.5)/(sigma**2) ) 
def gaussian_grad_2(x):
    return  gaussian(x) * (- (x[:,1:2] - 0.5)/(sigma**2) ) 
def gaussian_grad_3(x):
    return  gaussian(x) * (- (x[:,2:3] - 0.5)/(sigma**2) ) 

def u_exact(x):
    return gaussian(x) * torch.cos(2*pi*freq*x[:,0:1]) 
def alpha(x): 
    return torch.ones(x.size(0),1).to(device)

def u_grad_1(x):
    return  torch.cos(2*pi*freq*x[:,0:1]) *gaussian_grad_1(x) \
            - 2*pi*freq * torch.sin(2*pi*freq*x[:,0:1]) * gaussian(x) 
def u_grad_2(x):
    return torch.cos(2*pi*freq*x[:,0:1]) * gaussian_grad_2(x)
def u_grad_3(x):
    return  torch.cos(2*pi*freq*x[:,0:1]) * gaussian_grad_3(x)

def u_exact_grad():
    d = 3 
    def u_grad_1(x):
        return  torch.cos(2*pi*freq*x[:,0:1]) *gaussian_grad_1(x) \
                - 2*pi*freq * torch.sin(2*pi*freq*x[:,0:1]) * gaussian(x) 
    def u_grad_2(x):
        return torch.cos(2*pi*freq*x[:,0:1]) * gaussian_grad_2(x)
    def u_grad_3(x):
        return  torch.cos(2*pi*freq*x[:,0:1]) * gaussian_grad_3(x)

    u_grad=[u_grad_1, u_grad_2,u_grad_3] 
    return u_grad

def laplace_u_exact(x):
    return - 2*pi*freq * torch.sin(2*pi*freq*x[:,0:1]) *gaussian_grad_1(x) \
            + torch.cos(2*pi*freq*x[:,0:1])*( gaussian(x) * ( ((x[:,0:1] - 0.5)/(sigma**2))**2 -1/(sigma**2))  ) \
            -( (2*pi*freq)**2 * torch.cos(2*pi*freq*x[:,0:1]) * gaussian(x) + (2*pi*freq)*torch.sin(2*pi*freq*x[:,0:1]) * gaussian_grad_1(x) ) \
            + torch.cos(2*pi*freq*x[:,0:1]) * (gaussian(x) * ( ((x[:,1:2] - 0.5)/(sigma**2))**2 -1/(sigma**2) )  ) \
            + torch.cos(2*pi*freq*x[:,0:1]) * ( gaussian(x) * ( ((x[:,2:3] - 0.5)/(sigma**2))**2 -1/(sigma**2) )   ) \

def target(x):
    return - laplace_u_exact(x) + nonlinear(u_exact(x)) 

def g_N(dim):
    u_grad = u_exact_grad() 
    bcs_N = []
    for i in range(dim):
        bcs_N.append((i, u_grad[i]))
    return bcs_N

def u_exact_approx(x):
    return 0.99 * u_exact(x)

def rhs(x):
    return  -laplace_u_exact(x) + nonlinear(u_exact(x)) 


function_name = "gabor" 
filename_write = "3DCGA-{}-order.txt".format(function_name)
f_write = open(filename_write, "a")
f_write.write("\n")
f_write.close() 
save = True 

relu_k = 3
load_model_data = {'loadOrNot':True ,
                  'model_filename':"data-pb/model_OGA_3D_gabor_relu_3_neuron_512_N_384_randomized.pt",
                   'errl2_filename':'data-pb/errl2_OGA_3D_gabor_relu_3_neuron_512_N_384_randomized.pt',
                   'errh10_filename':'data-pb/errh10_OGA_3D_gabor_relu_3_neuron_512_N_384_randomized.pt',
                  'relu_k':3,
                  'neuron_num': 512}
for N_list in [[2**3,2**3,2**3]]: # ,[2**6,2**6],[2**7,2**7] 
    f_write = open(filename_write, "a")
    if load_model_data['loadOrNot'] == True: 
        my_model = model(3,load_model_data['neuron_num'],1,load_model_data['relu_k']).to(device)
        my_model.load_state_dict(torch.load(load_model_data['model_filename']))
    else: 
        my_model = None 
    Nx = 50
    order = 2
    exponent = 10
    num_epochs = (2**exponent) - load_model_data["neuron_num"] if load_model_data['loadOrNot'] else 2**exponent  
    plot_freq = num_epochs 
    N = np.prod(N_list)
    memomry_size = 2**30 
    err_QMC2, err_h10, my_model = CGANonlinearPoissonReLU3D(my_model,rhs,alpha, u_exact, u_exact_grad,g_N,\
                                        N_list,num_epochs,plot_freq, Nx, order, k = relu_k, \
                                        rand_deter = 'rand', linear_solver = "direct",memory = memomry_size)
    print(err_QMC2)
    print(err_h10)
    if load_model_data['loadOrNot']: # append the previous errrs 
        err_l2_previous = torch.load(load_model_data['errl2_filename'])
        err_h10_previous = torch.load(load_model_data['errh10_filename'])
        err_l2_combined = torch.zeros(2**exponent+1)
        err_h10_combined = torch.zeros(2**exponent+1)
        err_l2_combined[:load_model_data['neuron_num']+1] = err_l2_previous[:]
        err_l2_combined[load_model_data['neuron_num']+1 :] =  err_QMC2[1:]
        err_h10_combined[:load_model_data['neuron_num']+1] = err_h10_previous[:]
        err_h10_combined[load_model_data['neuron_num']+1 :] = err_h10[1:]
        err_QMC2 = err_l2_combined
        err_h10 = err_h10_combined 
    if save: 
        folder = 'data-pb/'
        neuron_num_save = load_model_data['neuron_num'] + num_epochs if load_model_data['loadOrNot'] else num_epochs
        filename = folder + 'errl2_OGA_3D_{}_relu_{}_neuron_{}_N_{}_randomized.pt'.format(function_name,relu_k,neuron_num_save,N)
        torch.save(err_QMC2,filename) 
        filename = folder + 'errh10_OGA_3D_{}_relu_{}_neuron_{}_N_{}_randomized.pt'.format(function_name,relu_k,neuron_num_save,N)
        torch.save(err_h10,filename) 
        filename = folder + 'model_OGA_3D_{}_relu_{}_neuron_{}_N_{}_randomized.pt'.format(function_name,relu_k,neuron_num_save,N)
        torch.save(my_model.state_dict(),filename)

    show_convergence_order(err_QMC2,err_h10,exponent,N,filename_write,True)
    show_convergence_order_latex(err_QMC2,err_h10,exponent,k =relu_k,d = 3)

using linear solver:  direct
epoch:  1	======TIME=======f and N(u) terms time : 0.0027904510498046875
======TIME=======stiffness matrix terms time : 0.04333353042602539
======TIME=======neumann bd terms time : 0.002124786376953125
=======> argmax subproblem time:  0.04854106903076172
assembling the matrix time taken:  0.0009076595306396484
solving Ax = b time taken:  0.12247443199157715
total size: 513 1000000 = 513000000
num batches:  1
newton iteration:  1
sol_update_l2_norm:7.3996045114237745 	 residual l2 norm: 5.100221031323618e-05 
newton iteration:  2
sol_update_l2_norm:0.0012292564692304626 	 residual l2 norm: 1.2066984239983582e-11 
newton iteration:  3
sol_update_l2_norm:4.313037564659111e-06 	 residual l2 norm: 2.556211501847181e-12 
converged at iteration:  3
sol_update_l2_norm:4.313037564659111e-06 	 residual l2 norm: 2.556211501847181e-12 
epoch:  2	======TIME=======f and N(u) terms time : 0.002566099166870117
======TIME=======stiffness matrix terms time : 0.0415246486663

assembling the matrix time taken:  0.2561969757080078
solving Ax = b time taken:  0.060782432556152344
total size: 523 1000000 = 523000000
num batches:  1
newton iteration:  1
sol_update_l2_norm:5.192089777434378 	 residual l2 norm: 4.400892483673619e-05 
newton iteration:  2
sol_update_l2_norm:0.00048074930277528405 	 residual l2 norm: 6.239859991461614e-09 
newton iteration:  3
sol_update_l2_norm:2.9528482973433453e-06 	 residual l2 norm: 1.577014378248071e-12 
converged at iteration:  3
sol_update_l2_norm:2.9528482973433453e-06 	 residual l2 norm: 1.577014378248071e-12 
epoch:  12	======TIME=======f and N(u) terms time : 0.0025594234466552734
======TIME=======stiffness matrix terms time : 0.04506325721740723
======TIME=======neumann bd terms time : 0.0017168521881103516
=======> argmax subproblem time:  0.049486637115478516
assembling the matrix time taken:  0.47027039527893066
solving Ax = b time taken:  0.05621147155761719
total size: 524 1000000 = 524000000
num batches:  1
newton

assembling the matrix time taken:  0.5880794525146484
solving Ax = b time taken:  0.059009552001953125
total size: 533 1000000 = 533000000
num batches:  1
newton iteration:  1
sol_update_l2_norm:17.15572824391222 	 residual l2 norm: 2.3045965703925213e-05 
newton iteration:  2
sol_update_l2_norm:0.004711549403891258 	 residual l2 norm: 4.447022320379019e-09 
newton iteration:  3
sol_update_l2_norm:2.6690684127462968e-06 	 residual l2 norm: 1.9167934458411273e-12 
converged at iteration:  3
sol_update_l2_norm:2.6690684127462968e-06 	 residual l2 norm: 1.9167934458411273e-12 
epoch:  22	======TIME=======f and N(u) terms time : 0.0025835037231445312
======TIME=======stiffness matrix terms time : 0.04529929161071777
======TIME=======neumann bd terms time : 0.001737356185913086
=======> argmax subproblem time:  0.04976940155029297
assembling the matrix time taken:  0.3714172840118408
solving Ax = b time taken:  0.07105445861816406
total size: 534 1000000 = 534000000
num batches:  1
newton i

assembling the matrix time taken:  0.2980670928955078
solving Ax = b time taken:  0.05954289436340332
total size: 543 1000000 = 543000000
num batches:  2
newton iteration:  1
sol_update_l2_norm:23.44728876028796 	 residual l2 norm: 5.2279829001388544e-05 
newton iteration:  2
sol_update_l2_norm:0.0018042951811587764 	 residual l2 norm: 1.5738437385183695e-09 
newton iteration:  3
sol_update_l2_norm:4.098204036145432e-06 	 residual l2 norm: 2.1455172712302415e-12 
converged at iteration:  3
sol_update_l2_norm:4.098204036145432e-06 	 residual l2 norm: 2.1455172712302415e-12 
epoch:  32	======TIME=======f and N(u) terms time : 0.0025436878204345703
======TIME=======stiffness matrix terms time : 0.04749155044555664
======TIME=======neumann bd terms time : 0.0017197132110595703
=======> argmax subproblem time:  0.051904916763305664
assembling the matrix time taken:  0.127701997756958
solving Ax = b time taken:  0.0591890811920166
total size: 544 1000000 = 544000000
num batches:  2
newton it

assembling the matrix time taken:  0.30037522315979004
solving Ax = b time taken:  0.06003451347351074
total size: 553 1000000 = 553000000
num batches:  2
newton iteration:  1
sol_update_l2_norm:13.621701057810489 	 residual l2 norm: 8.977887679378141e-06 
newton iteration:  2
sol_update_l2_norm:0.0007672521224570966 	 residual l2 norm: 8.995222950893676e-10 
newton iteration:  3
sol_update_l2_norm:5.451464662951044e-06 	 residual l2 norm: 1.2147168267421757e-12 
converged at iteration:  3
sol_update_l2_norm:5.451464662951044e-06 	 residual l2 norm: 1.2147168267421757e-12 
epoch:  42	======TIME=======f and N(u) terms time : 0.002580404281616211
======TIME=======stiffness matrix terms time : 0.04771876335144043
======TIME=======neumann bd terms time : 0.0017192363739013672
=======> argmax subproblem time:  0.05218243598937988
assembling the matrix time taken:  0.29982590675354004
solving Ax = b time taken:  0.060028076171875
total size: 554 1000000 = 554000000
num batches:  2
newton ite

...

epoch:  470	======TIME=======f and N(u) terms time : 0.002518892288208008
======TIME=======stiffness matrix terms time : 0.06103634834289551
======TIME=======neumann bd terms time : 0.001857757568359375
=======> argmax subproblem time:  0.06559491157531738
assembling the matrix time taken:  0.8130645751953125
solving Ax = b time taken:  0.1551210880279541
total size: 982 1000000 = 982000000
num batches:  2
newton iteration:  1
sol_update_l2_norm:10.922565777605785 	 residual l2 norm: 3.992229897331619e-06 
newton iteration:  2
sol_update_l2_norm:0.00917348811383024 	 residual l2 norm: 5.6752170268410076e-11 
newton iteration:  3
sol_update_l2_norm:6.681973638135204e-05 	 residual l2 norm: 2.7970101472642614e-12 
converged at iteration:  3
sol_update_l2_norm:6.681973638135204e-05 	 residual l2 norm: 2.7970101472642614e-12 
epoch:  471	======TIME=======f and N(u) terms time : 0.0025370121002197266
======TIME=======stiffness matrix terms time : 0.0628807544708252
======TIME=======neumann 

epoch:  480	======TIME=======f and N(u) terms time : 0.0025207996368408203
======TIME=======stiffness matrix terms time : 0.06100964546203613
======TIME=======neumann bd terms time : 0.0018024444580078125
=======> argmax subproblem time:  0.06550931930541992
assembling the matrix time taken:  0.8212766647338867
solving Ax = b time taken:  0.15425777435302734
total size: 992 1000000 = 992000000
num batches:  2
newton iteration:  1
sol_update_l2_norm:8.094643650862025 	 residual l2 norm: 2.5262546452723473e-06 
newton iteration:  2
sol_update_l2_norm:0.013283108432322391 	 residual l2 norm: 1.7038679643390238e-11 
newton iteration:  3
sol_update_l2_norm:0.00010687492818353824 	 residual l2 norm: 5.926135916162105e-13 
converged at iteration:  3
sol_update_l2_norm:0.00010687492818353824 	 residual l2 norm: 5.926135916162105e-13 
epoch:  481	======TIME=======f and N(u) terms time : 0.0025136470794677734
======TIME=======stiffness matrix terms time : 0.06309032440185547
======TIME=======neu

epoch:  490	======TIME=======f and N(u) terms time : 0.0025529861450195312
======TIME=======stiffness matrix terms time : 0.061249494552612305
======TIME=======neumann bd terms time : 0.001806020736694336
=======> argmax subproblem time:  0.06580114364624023
assembling the matrix time taken:  0.8326129913330078
solving Ax = b time taken:  0.1562356948852539
total size: 1002 1000000 = 1002000000
num batches:  2
newton iteration:  1
sol_update_l2_norm:29.363176292641786 	 residual l2 norm: 7.9564261012181e-06 
newton iteration:  2
sol_update_l2_norm:0.1010507391397491 	 residual l2 norm: 6.495216153189272e-10 
newton iteration:  3
sol_update_l2_norm:0.00028791018011623155 	 residual l2 norm: 9.116081051266935e-13 
converged at iteration:  3
sol_update_l2_norm:0.00028791018011623155 	 residual l2 norm: 9.116081051266935e-13 
epoch:  491	======TIME=======f and N(u) terms time : 0.0025522708892822266
======TIME=======stiffness matrix terms time : 0.06351900100708008
======TIME=======neumann

======TIME=======stiffness matrix terms time : 0.061540842056274414
======TIME=======neumann bd terms time : 0.0018582344055175781
=======> argmax subproblem time:  0.06610465049743652
assembling the matrix time taken:  0.8392786979675293
solving Ax = b time taken:  0.1557326316833496
total size: 1012 1000000 = 1012000000
num batches:  2
newton iteration:  1
sol_update_l2_norm:4.927934429054407 	 residual l2 norm: 1.775043002041838e-05 
newton iteration:  2
sol_update_l2_norm:0.014150323555010731 	 residual l2 norm: 3.462191199814501e-11 
newton iteration:  3
sol_update_l2_norm:7.297575987135402e-05 	 residual l2 norm: 2.662930055148966e-12 
converged at iteration:  3
sol_update_l2_norm:7.297575987135402e-05 	 residual l2 norm: 2.662930055148966e-12 
epoch:  501	======TIME=======f and N(u) terms time : 0.0025081634521484375
======TIME=======stiffness matrix terms time : 0.06369924545288086
======TIME=======neumann bd terms time : 0.0018725395202636719
=======> argmax subproblem time:  

epoch:  510	======TIME=======f and N(u) terms time : 0.0025358200073242188
======TIME=======stiffness matrix terms time : 0.06205868721008301
======TIME=======neumann bd terms time : 0.0018608570098876953
=======> argmax subproblem time:  0.06663060188293457
assembling the matrix time taken:  0.848132848739624
solving Ax = b time taken:  0.15658140182495117
total size: 1022 1000000 = 1022000000
num batches:  2
newton iteration:  1
sol_update_l2_norm:4.448815311916088 	 residual l2 norm: 2.041101341289069e-05 
newton iteration:  2
sol_update_l2_norm:0.0028410512384542366 	 residual l2 norm: 2.234260829891003e-10 
newton iteration:  3
sol_update_l2_norm:1.3100587694104663e-05 	 residual l2 norm: 8.816545755859713e-13 
converged at iteration:  3
sol_update_l2_norm:1.3100587694104663e-05 	 residual l2 norm: 8.816545755859713e-13 
epoch:  511	======TIME=======f and N(u) terms time : 0.00255584716796875
======TIME=======stiffness matrix terms time : 0.06447052955627441
======TIME=======neuma

In [20]:
folder = 'data-pb/'
neuron_num_save = load_model_data['neuron_num'] + num_epochs if load_model_data['loadOrNot'] else num_epochs
filename = folder + 'errl2_OGA_3D_{}_relu_{}_neuron_{}_N_{}_randomized.pt'.format(function_name,relu_k,neuron_num_save,N)
torch.save(err_QMC2,filename) 
filename = folder + 'errh10_OGA_3D_{}_relu_{}_neuron_{}_N_{}_randomized.pt'.format(function_name,relu_k,neuron_num_save,N)
torch.save(err_h10,filename) 
filename = folder + 'model_OGA_3D_{}_relu_{}_neuron_{}_N_{}_randomized.pt'.format(function_name,relu_k,neuron_num_save,N)
torch.save(my_model.state_dict(),filename)

In [22]:
err_l2_512 = torch.load(folder + 'errl2_OGA_3D_{}_relu_{}_neuron_{}_N_{}_randomized.pt'.format(function_name,relu_k,neuron_num_save,N))
err_h10_512 = torch.load(folder + 'errh10_OGA_3D_{}_relu_{}_neuron_{}_N_{}_randomized.pt'.format(function_name,relu_k,neuron_num_save,N))
show_convergence_order_latex(err_QMC2,err_h10,exponent,k =relu_k,d = 3)

neuron num  & 	 $\|u-u_n \|_{L^2}$ & 	 order $O(n^{-1.67})$ & 	 $ | u -u_n |_{H^1}$ & 	 order $O(n^{-1.33})$ \\ \hline \hline 
4 		 & 0.097603 &		 * & 		 2.228954 & 		 *  \\ \hline  

8 		 &  9.692e-02 &  		 0.01 &  		 2.228e+00 &  		 0.00 \\ \hline  

16 		 &  9.213e-02 &  		 0.07 &  		 2.208e+00 &  		 0.01 \\ \hline  

32 		 &  8.175e-02 &  		 0.17 &  		 2.080e+00 &  		 0.09 \\ \hline  

64 		 &  5.566e-02 &  		 0.55 &  		 1.647e+00 &  		 0.34 \\ \hline  

128 		 &  2.573e-02 &  		 1.11 &  		 1.315e+00 &  		 0.32 \\ \hline  

256 		 &  8.012e-03 &  		 1.68 &  		 6.471e-01 &  		 1.02 \\ \hline  

512 		 &  1.891e-03 &  		 2.08 &  		 1.716e-01 &  		 1.91 \\ \hline  



## Test cosine function 

In [32]:
freq = 2 
def u_exact(x):
    return torch.cos(freq*pi*x[:,0:1])*torch.cos( freq*pi*x[:,1:2]) * torch.cos(freq*pi*x[:,2:3])  
def alpha(x): 
    return torch.ones(x.size(0),1).to(device)

def u_exact_grad():
    d = 3 
    def grad_1(x):
        return - freq*pi* torch.sin(freq*pi*x[:,0:1])*torch.cos( freq*pi*x[:,1:2]) * torch.cos(freq*pi*x[:,2:3])   
    def grad_2(x):
        return - freq*pi* torch.cos(freq*pi*x[:,0:1])*torch.sin( freq*pi*x[:,1:2]) * torch.cos(freq*pi*x[:,2:3])  
    def grad_3(x):
        return - freq*pi* torch.cos(freq*pi*x[:,0:1])*torch.cos( freq*pi*x[:,1:2]) * torch.sin(freq*pi*x[:,2:3])   
    
    u_grad=[grad_1, grad_2,grad_3] 

    return u_grad
def laplace_u_exact(x):
    return -3*(freq*pi)**2 * torch.cos(freq*pi*x[:,0:1])*torch.cos( freq*pi*x[:,1:2]) * torch.cos(freq*pi*x[:,2:3])
# def target(x):
#     z = -laplace_u_exact(x) + u_exact(x)**3 
#     return z 

def u_exact_approx(x):
    return 0.7 * u_exact(x)

def rhs(x):
    return  -laplace_u_exact(x) + nonlinear(u_exact(x))

g_N = None 


function_name = "cos4pix" 
filename_write = "3DCGA-{}-order.txt".format(function_name)
f_write = open(filename_write, "a")
f_write.write("\n")
f_write.close() 
save = False 
relu_k = 3
for N_list in [[2*3,2**3,2**3]]: # ,[2**6,2**6],[2**7,2**7] 
    # save = True 
    f_write = open(filename_write, "a")
    my_model = None 
    Nx = 50
    order = 3
    exponent = 7
    num_epochs = 2**exponent  
    plot_freq = num_epochs 
    N = np.prod(N_list)
    err_QMC2, err_h10, my_model = CGANonlinearPoissonReLU3D(my_model,rhs,alpha, u_exact, u_exact_grad,g_N, N_list,num_epochs,plot_freq, Nx, order, k = relu_k, rand_deter = 'rand', linear_solver = "direct")
    if save: 
        folder = 'data-neumann/'
        filename = folder + 'err_OGA_2D_{}_neuron_{}_N_{}_deterministic.pt'.format(function_name,num_epochs,N)
        torch.save(err_QMC2,filename) 
        folder = 'data-neumann/'
        filename = folder + 'model_OGA_2D_{}_neuron_{}_N_{}_deterministic.pt'.format(function_name,num_epochs,N)
        torch.save(my_model,filename)

    show_convergence_order(err_QMC2,err_h10,exponent,N,filename_write,False)
    show_convergence_order_latex(err_QMC2,err_h10,exponent,k =relu_k,d = 3)

using linear solver:  direct
epoch:  1	======TIME=======f and u^3 terms time : 0.00040221214294433594
======TIME=======stiffness matrix terms time : 9.5367431640625e-07
======TIME=======neumann bd terms time : 2.384185791015625e-07
=======> argmax subproblem time:  0.000537872314453125
total size: 1 1000000 = 1000000
num batches:  1
newton iteration:  1
sol_update_l2_norm:0.22455818462385624 	 residual l2 norm: 0.022839427539190275 
newton iteration:  2
sol_update_l2_norm:5.874028747627114e-06 	 residual l2 norm: 5.974843031962551e-07 
newton iteration:  3
sol_update_l2_norm:1.2049027379020042e-14 	 residual l2 norm: 1.2255821357776142e-15 
converged at iteration:  3
sol_update_l2_norm:1.2049027379020042e-14 	 residual l2 norm: 1.2255821357776142e-15 
epoch:  2	======TIME=======f and u^3 terms time : 0.0011060237884521484
======TIME=======stiffness matrix terms time : 0.01655411720275879
======TIME=======neumann bd terms time : 7.152557373046875e-07
=======> argmax subproblem time:  0.

assembling the matrix time taken:  0.00018143653869628906
solving Ax = b time taken:  0.0008037090301513672
total size: 12 1000000 = 12000000
num batches:  1
newton iteration:  1
sol_update_l2_norm:0.29456007169800663 	 residual l2 norm: 0.11534735216379899 
newton iteration:  2
sol_update_l2_norm:1.0135690649990655e-05 	 residual l2 norm: 5.305637885335858e-05 
newton iteration:  3
sol_update_l2_norm:2.334639919494561e-13 	 residual l2 norm: 9.319451047539626e-13 
converged at iteration:  3
sol_update_l2_norm:2.334639919494561e-13 	 residual l2 norm: 9.319451047539626e-13 
epoch:  13	======TIME=======f and u^3 terms time : 0.001119375228881836
======TIME=======stiffness matrix terms time : 0.01729297637939453
======TIME=======neumann bd terms time : 9.5367431640625e-07
=======> argmax subproblem time:  0.01856851577758789
assembling the matrix time taken:  0.00017070770263671875
solving Ax = b time taken:  0.0008883476257324219
total size: 13 1000000 = 13000000
num batches:  1
newton 

sol_update_l2_norm:1.2622169341693181 	 residual l2 norm: 0.014578077803039395 
newton iteration:  2
sol_update_l2_norm:5.86630372878789e-05 	 residual l2 norm: 2.2558632331597758e-05 
newton iteration:  3
sol_update_l2_norm:1.49915314330398e-12 	 residual l2 norm: 6.650597192962242e-13 
converged at iteration:  3
sol_update_l2_norm:1.49915314330398e-12 	 residual l2 norm: 6.650597192962242e-13 
epoch:  24	======TIME=======f and u^3 terms time : 0.0011239051818847656
======TIME=======stiffness matrix terms time : 0.01761341094970703
======TIME=======neumann bd terms time : 4.76837158203125e-07
=======> argmax subproblem time:  0.018893003463745117
assembling the matrix time taken:  0.00016880035400390625
solving Ax = b time taken:  0.0014629364013671875
total size: 24 1000000 = 24000000
num batches:  1
newton iteration:  1
sol_update_l2_norm:5.377628506381402 	 residual l2 norm: 0.03813389241198335 
newton iteration:  2
sol_update_l2_norm:0.00048816580289643986 	 residual l2 norm: 0.00

======TIME=======stiffness matrix terms time : 0.02094411849975586
======TIME=======neumann bd terms time : 9.5367431640625e-07
=======> argmax subproblem time:  0.022475004196166992
assembling the matrix time taken:  0.00018334388732910156
solving Ax = b time taken:  0.0020928382873535156
total size: 34 1000000 = 34000000
num batches:  1
newton iteration:  1
sol_update_l2_norm:60.2701821396901 	 residual l2 norm: 0.025958526264395633 
newton iteration:  2
sol_update_l2_norm:0.005648579052254733 	 residual l2 norm: 0.0010747497769728821 
newton iteration:  3
sol_update_l2_norm:2.632924517151843e-09 	 residual l2 norm: 2.5771335067215187e-09 
newton iteration:  4
sol_update_l2_norm:2.234773640208201e-12 	 residual l2 norm: 1.2027672229492393e-12 
converged at iteration:  4
sol_update_l2_norm:2.234773640208201e-12 	 residual l2 norm: 1.2027672229492393e-12 
epoch:  35	======TIME=======f and u^3 terms time : 0.00119781494140625
======TIME=======stiffness matrix terms time : 0.020855188369

assembling the matrix time taken:  0.00017118453979492188
solving Ax = b time taken:  0.0024764537811279297
total size: 44 1000000 = 44000000
num batches:  1
newton iteration:  1
sol_update_l2_norm:24.313266875684864 	 residual l2 norm: 0.010804695521924862 
newton iteration:  2
sol_update_l2_norm:0.001596613598874812 	 residual l2 norm: 4.277684687634604e-05 
newton iteration:  3
sol_update_l2_norm:1.8076875415444906e-11 	 residual l2 norm: 1.4422585109290618e-11 
converged at iteration:  3
sol_update_l2_norm:1.8076875415444906e-11 	 residual l2 norm: 1.4422585109290618e-11 
epoch:  45	======TIME=======f and u^3 terms time : 0.0011136531829833984
======TIME=======stiffness matrix terms time : 0.02120685577392578
======TIME=======neumann bd terms time : 4.76837158203125e-07
=======> argmax subproblem time:  0.022485971450805664
assembling the matrix time taken:  0.00017142295837402344
solving Ax = b time taken:  0.002539396286010742
total size: 45 1000000 = 45000000
num batches:  1
new

...

assembling the matrix time taken:  0.00018525123596191406
solving Ax = b time taken:  0.00598454475402832
total size: 84 1000000 = 84000000
num batches:  1
newton iteration:  1
sol_update_l2_norm:56.985025260332485 	 residual l2 norm: 0.0013020834069167896 
newton iteration:  2
sol_update_l2_norm:0.0026913974592082867 	 residual l2 norm: 0.00036389118701261257 
newton iteration:  3
sol_update_l2_norm:3.2784719847582563e-10 	 residual l2 norm: 1.274373578652883e-10 
converged at iteration:  3
sol_update_l2_norm:3.2784719847582563e-10 	 residual l2 norm: 1.274373578652883e-10 
epoch:  85	======TIME=======f and u^3 terms time : 0.0011734962463378906
======TIME=======stiffness matrix terms time : 0.02217578887939453
======TIME=======neumann bd terms time : 9.5367431640625e-07
=======> argmax subproblem time:  0.023543834686279297
assembling the matrix time taken:  0.0001747608184814453
solving Ax = b time taken:  0.006085872650146484
total size: 85 1000000 = 85000000
num batches:  1
newton

assembling the matrix time taken:  0.00018334388732910156
solving Ax = b time taken:  0.006282329559326172
total size: 94 1000000 = 94000000
num batches:  1
newton iteration:  1
sol_update_l2_norm:73.75897056313721 	 residual l2 norm: 0.0007095905115045528 
newton iteration:  2
sol_update_l2_norm:0.0005352456277098922 	 residual l2 norm: 1.7095691781653254e-05 
newton iteration:  3
sol_update_l2_norm:1.565733899731963e-11 	 residual l2 norm: 6.7469744929790195e-12 
converged at iteration:  3
sol_update_l2_norm:1.565733899731963e-11 	 residual l2 norm: 6.7469744929790195e-12 
epoch:  95	======TIME=======f and u^3 terms time : 0.001142740249633789
======TIME=======stiffness matrix terms time : 0.022582292556762695
======TIME=======neumann bd terms time : 4.76837158203125e-07
=======> argmax subproblem time:  0.023892641067504883
assembling the matrix time taken:  0.0001697540283203125
solving Ax = b time taken:  0.006402015686035156
total size: 95 1000000 = 95000000
num batches:  1
newto

sol_update_l2_norm:0.0009063262584987207 	 residual l2 norm: 2.6660192408452312e-05 
newton iteration:  3
sol_update_l2_norm:2.550581834341054e-11 	 residual l2 norm: 2.7342512939775136e-12 
converged at iteration:  3
sol_update_l2_norm:2.550581834341054e-11 	 residual l2 norm: 2.7342512939775136e-12 
epoch:  105	======TIME=======f and u^3 terms time : 0.0011627674102783203
======TIME=======stiffness matrix terms time : 0.022853374481201172
======TIME=======neumann bd terms time : 4.76837158203125e-07
=======> argmax subproblem time:  0.024186134338378906
assembling the matrix time taken:  0.00017833709716796875
solving Ax = b time taken:  0.0066416263580322266
total size: 105 1000000 = 105000000
num batches:  1
newton iteration:  1
sol_update_l2_norm:26.250955107711057 	 residual l2 norm: 0.0005295455835219796 
newton iteration:  2
sol_update_l2_norm:0.0002799964122903308 	 residual l2 norm: 1.4147047307398476e-05 
newton iteration:  3
sol_update_l2_norm:8.373295824764386e-11 	 residu

sol_update_l2_norm:3.234925659219173e-11 	 residual l2 norm: 2.0836083009019937e-12 
converged at iteration:  3
sol_update_l2_norm:3.234925659219173e-11 	 residual l2 norm: 2.0836083009019937e-12 
epoch:  115	======TIME=======f and u^3 terms time : 0.0011730194091796875
======TIME=======stiffness matrix terms time : 0.02632284164428711
======TIME=======neumann bd terms time : 9.5367431640625e-07
=======> argmax subproblem time:  0.027664899826049805
assembling the matrix time taken:  0.000164031982421875
solving Ax = b time taken:  0.00710749626159668
total size: 115 1000000 = 115000000
num batches:  1
newton iteration:  1
sol_update_l2_norm:53.125904141055635 	 residual l2 norm: 0.0003677357842337922 
newton iteration:  2
sol_update_l2_norm:0.0003423537001758833 	 residual l2 norm: 9.519780111217083e-06 
newton iteration:  3
sol_update_l2_norm:3.000434476408167e-11 	 residual l2 norm: 6.558981946351738e-13 
converged at iteration:  3
sol_update_l2_norm:3.000434476408167e-11 	 residual

sol_update_l2_norm:32.69839699539918 	 residual l2 norm: 0.0003233713889806698 
newton iteration:  2
sol_update_l2_norm:6.368856460884515e-05 	 residual l2 norm: 1.5908471415485726e-05 
newton iteration:  3
sol_update_l2_norm:3.9185433273019695e-11 	 residual l2 norm: 1.887328297573768e-12 
converged at iteration:  3
sol_update_l2_norm:3.9185433273019695e-11 	 residual l2 norm: 1.887328297573768e-12 
epoch:  125	======TIME=======f and u^3 terms time : 0.0011591911315917969
======TIME=======stiffness matrix terms time : 0.023413896560668945
======TIME=======neumann bd terms time : 4.76837158203125e-07
=======> argmax subproblem time:  0.024892807006835938
assembling the matrix time taken:  0.00017333030700683594
solving Ax = b time taken:  0.007532596588134766
total size: 125 1000000 = 125000000
num batches:  1
newton iteration:  1
sol_update_l2_norm:14.05352694431333 	 residual l2 norm: 0.000387807741857618 
newton iteration:  2
sol_update_l2_norm:2.611207878973738e-05 	 residual l2 no

In [15]:
N_list = [2**3,2**3,2**3]
N = np.prod(N_list)
N

512

In [5]:

load_model_data = {'loadOrNot':False,
                  'model_filename':"data-pb/xxx",
                  'relu_k':3,
                  'neuron_num': 128}


False

In [21]:
err = torch.load('data-pb/err_OGA_3D_gabor_relu_3_neuron_128_N_384_randomized.pt')
err.size()

torch.Size([129])